<a href="https://colab.research.google.com/github/Meaveryway/Backend_AntTech/blob/master/V_Det_Plagiarism_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
from dataclasses import dataclass, field #to reduce boilerplate code
from typing import List, Tuple, Union
from enum import Enum
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle #To highlight on graphs
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import torch
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# GPU activate/check
import torch
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device

# Dataframes en mode interactif
%load_ext google.colab.data_table
# %unload_ext google.colab.data_table

# Profiling
!pip install memory_profiler
%load_ext memory_profiler

!pip install icecream
from icecream import ic

!pip install langdetect
from langdetect import detect

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [ ]:
#Display local distro
!head -n1 /etc/issue

Ubuntu 18.04.5 LTS \n \l


##Hyperparameters

In [2]:
NGRAM_SIZE_COMMON_SUBSTRING = 3
STRIDE_COMMON_SUBSTRING = 1
SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING = 5

NGRAM_SIZE_SHINGLES = 15
STRIDE_SHINGLES = 7
SKIP_TOLERANCE = int(NGRAM_SIZE_SHINGLES/STRIDE_SHINGLES)
THRESHOLD_JACCARD = 0.25
THRESHOLD_FASTTEXT = 0.11

In [ ]:
#Sample typical docstring & typing
def toColumnNamesFixedLen(
    oX: pd.DataFrame,
    column_length: int = 3,
    column_separator: str = "_",
    inplace: bool = True,
    verbose: bool = True,
) -> pd.DataFrame:
    """
        Truncate column name to a specific length.  If column length
        is shorter, then column length left as is.

        This method mutates the original DataFrame.

        Method chaining will truncate all columns to a given length
        and append a given separator character with the index of
        duplicate columns, except for the first distinct column name.

        Parameters:
            X: dataset

        Keywords:
            column_length: 3 (default)
                Truncate all columns names to length.
                The column separator value and number for duplicate
                column name does not contribute to total string
                length. If all columns string lenths are truncated
                to 10 characters, the first distinct column will
                be 10 characters and the remaining columns are
                12 characters with a column separator of one
                character).

            column_separator: "_" (default)
                The separator to append plus incremental Int to
                create unique column names. Care should be 
                taken in choosing non-default str so as to
                create legal pandas column
                names.

            verbose: True (default)
                True: output
                False: silent

            inplace: True  (default)
                True: replace 1st argument with resulting dataframe
                False:  (boolean)change unplace the dataframe X

        Returns: A pandas DataFrame (pd.DataFrame) with truncated
        column lengths.
    """
    #reST format to use
    """
    This is a reST style.

    :param param1: this is a first param
    :param param2: this is a second param
    :returns: this is a description of what is returned
    :raises keyError: raises an exception
    """

In [ ]:
# use np.ndarray instead of list, init with None, then post_init:
#def __post_init__(self):
 #       self.outer_folds = dict()
  #      self.inner_folds = dict()

In [3]:
class Language(Enum):
  '''Language, to be detected from suspect document and used to load appropriate list of stop-words and fastText pre-trained model'''
  ARABIC = 'ar'
  FRENCH = 'fr'
  ENGLISH = 'en'

def detect_language(text: str) -> Language:
  '''DOCUMENT ME'''
  lang = detect(text.lower().strip())
  if lang in set(item.value for item in Language):
    return Language(lang)
  else:
    raise Exception("Error {language detection}: detected language is not among [ar, fr, en]: {}".format(lang))

#Report data structure

|Class|Description|content|
|---|---|---|
|**PlagReport**|a global report between suspect all retrieved candidates (an agg of sub_reports)| instances: List [(plag_instance, can_src)]<br> scores: DetectionScores<br> sus_fulltext: str<br>---<br> timestamp: timestamp <br> report_id: str <br> sus_name: str<br> sus_id: str <br> params: DetectParams|
|**SubReport**|a Jaccard-FastText global report between suspect and a single candidate only|sus_name: str <br> sus_id: str <br> can_src: CanSource <br> scores: DetectionScores<br> instances: List [plag_instance]|
|**PlagInstance**|a single plagiarism instance passage between suspect and a candidate|posrange_suspect: [int, int] <br> posrange_candidate: (int, int) <br> passsage_suspect: str <br> passsage_candidate: str <br> plag_type: Enum [PlagType]|
|**CanSource**|source candidate document|can_name: str <br> can_url: str <br> can_id: str <br> can_type: Enum [CanType]| 
|**DetectParams**|hyperparameters used to conduct comparisons|jaccard_threshold: float <br> jaccard_fasttext_threshold: float <br> fasttext_threshold: float <br> ngram_size: int <br> stride_size: int|
|**DetectOperation**||sus_name: str <br> sus_id:str <br> user_id: str <br> report_id: str|
|**DetectionScores**|to hold detailed and global percentages|score_syntac: float <br> score_semant:float <br> score_global: float |


---
|Enum|Properties|Description|
|---|---|---|
|**PlagType**|syntac <br> syntac_cls <br> semant <br> semant_cls|syntactic: pure Jaccard <br> syntactic with a close match: Jaccard with a skip_stride in the text passage <br> semantic: pure Fasttext <br> semantic with a close match: Fasttext with a skip_stride in the text passage|
|**CanType**|web_page <br> web_file <br> local_file <br> theses_file| page scraped from a search engine <br> file collected from a search engine <br> file collected from the user's DB <br> file collected from the theses-algerie DB|

In [4]:
class PlagType(Enum):
  '''Plagiarism type, to be associated with each plagiarism instance (PlagInstance).'''
  SYNTACT = 0
  SYNTACT_CLS = 1
  SEMANT = 2
  SEMANT_CLS = 3

class CanType(Enum):
  '''Candidate source type, to be associated with each candidate source (CanSource)'''
  WEB_PAGE = 0
  WEB_FILE = 1
  LOCAL_FILE = 2
  THESES_FILE = 3

In [5]:
@dataclass(frozen=True)
class DetectParams:
  """
  A class to hold detection parameters used to carry out a detection operation, which can be 
  used back to tokenize the source text and get the same results (so it's more for trackability).
  
  Attributes:
      jaccard_threshold:
          Used to determine the Jaccard distance under which two pairs are syntactically similar.
      jaccard_fasttext_threshold:
          Used to determine the distance percentage under which two pairs are to be passed to the semantic semalirty test.
          is > jaccard_threshold. if == 1 then any pair that isn't tagged as syntactically similar will be tested for semantic similarity.
      fasttext_threshold:
          Used to determine the cosine distance under which two pairs are semantically similar, by using their FasTtext features.
      ngram_size, stride_size:
          Used to tokenize text for both syntactic and semantic analysis
  """
  jaccard_threshold: float = 0.25
  jaccard_fasttext_threshold: float = 1.0
  fasttext_threshold: float = 0.11
  ngram_size: int = 15
  stride_size: int = 7

@dataclass(frozen=True)
class DetectionScores:
  """
  A class to hold detection scores registered after a comparison operation with a single candidate, or the aggregation of multiple reports.
  
  Attributes:
      syntac_score:
          Global syntactic similarity between suspect file and candidates (one if SubReport, multiple if PlagReport)
          syntac_score = size(suspect text) / sum(size (suspect segments that appear as a member in pairs with Jaccard similarity < threshold))
      semant_score:
          Global semantic similarity between suspect file and candidates (one if SubReport, multiple if PlagReport)
          semant_score= size(suspect text) / sum(size (suspect segments that appear as a member in pairs with FastText cosine similarity < threshold))
      global_score:
          Global similarity, calculated as merging score of both syntactic and semantic similarity (not their sum)
  """
  syntac_score: float
  semant_score:float
  global_score: float

@dataclass(frozen=True)
class CanSource:
  can_name: str
  can_url: str
  can_id: str
  can_type: CanType 

@dataclass(frozen=True)
class PlagInstance:
  posrange_suspect: (int, int)
  posrange_candidate: (int, int)
  passsage_suspect: str
  passsage_candidate: str
  plag_type: PlagType

@dataclass()
class SubReport:
  sus_name: str
  sus_id: str
  can_src: CanSource
  scores: DetectionScores
  instances: List[PlagInstance]

@dataclass()
class PlagReport:
  sub_reports: List[SubReport]
  sus_fulltext: str
  timestamp: int
  report_id: str
  sus_name: str
  sus_id: str
  params: DetectParams
  scores: DetectionScores = field(init=False)

  def __post_init__(self):
    object.__setattr__(self, 'scores', (aggregate_scores(self, self.sub_reports)))



In [6]:
now_stamp = time.time()
date = datetime.utcfromtimestamp(now_stamp).strftime('%Y-%m-%d %H:%M:%S')
date

'2021-07-03 19:01:40'

In [ ]:
candidate_1 = CanSource('wikipedia', 'wikipedia.org', '41498502fs', CanType.WEB_PAGE)
candidate_2 = CanSource('these de doctorat', 'theses-algerie.com', '414dqrw02fs', CanType.THESES_FILE)
params = DetectParams()

plaginstance1_1 = PlagInstance((3, 54), (17, 68), "this is some sort of suspect text", "this is some sort of candidate text", PlagType.SYNTACT)
plaginstance1_2 = PlagInstance((12, 14), (45, 47), "this is some sort of suspect text", "this is a kind of candidate passage", PlagType.SEMANT)
plaginstances1 = [plaginstance1_1, plaginstance1_2]

plaginstance2_1 = PlagInstance((3, 7), (31, 35), "this is some sort of secondary suspect text", "this is some sort of non-primary candidate text", PlagType.SYNTACT)
plaginstance2_2 = PlagInstance((23, 24), (57, 58), "this is some sort of secondary suspect text", "this is a kind of candidate passage that comes second", PlagType.SEMANT)
plaginstances2 = [plaginstance2_1, plaginstance2_2]

scores_1 = DetectionScores(.45, .47, .56)
scores_2 = DetectionScores(.35, .5, .35)

subreport_1 = SubReport("Suspect.pdf", "19381098da", candidate_1, scores_1, plaginstances1)
subreport_2 = SubReport("Suspect.pdf", "19381098da", candidate_2, scores_2, plaginstances2)

now_stamp = int(time.time())
sus_text = "this is some sort of suspect text this is some sort of suspect text this is some sort of secondary suspect text this is some sort of secondary suspect text"
global_report = PlagReport([subreport_1, subreport_2], sus_text, now_stamp, "149294fd", 'Suspect.pdf', '19381098da', params)



In [ ]:
global_report.scores

In [14]:
PlagReport.__doc__

'PlagReport(sub_reports: List[__main__.SubReport], sus_fulltext: str, timestamp: int, report_id: str, sus_name: str, sus_id: str, params: __main__.DetectParams)'

####Sample

In [13]:
samp_suspect = ["The significance of export-platform FDI is reported in a think about by Hanson, Mataloni, and Butcher (2001). Utilising information on the remote operations of US multinationals, they report that in spite of the fact that the normal share of trades in partner deals has remained steady at approximately one-third, there has been a significant increment in Mexico and Canada after the arrangement of NAFTA. Their econometric examination recommends that trade stage FDI is advanced by moo host-country exchange boundaries and disheartened by expansive host-country markets.Within the past decade, the worldwide commercial center has experienced the democratisation of extravagance, driven by a continually developing optimistic customer portion. This prevalent frame of unused extravagance (Silverstein and Fiske 2003a) centers on human association, self- expression, unlimited dissemination, and acknowledgment from others. Unused extravagance, too alluded to as “masstige” (i.e., mass glory; Cornell 2002), challenges routine convictions around extravagance showcasing and branding, counting estimating and cost focuses, brand expansions, buyer request, modernity, showcase steadiness, and the time required to reach the center lesson. The promoting of extravagance has contributed altogether to the democratisation of extravagance, but its impact truly is auxiliary to the impact of changing utilisation designs.The primary case we consider includes symmetric exchange costs on all joins, so that the two firms will each embrace the same number of plants, and either both or not one or the other will have a plant in S and, when they do, the trade designs of those plants will be symmetric. Third-country trade-stage generation emerges when moo generation costs in S lead a firm to utilise S to serve the other N nation, but the investment funds on settled costs from closing the domestic plant don't balanced the costs of shipping components to S and last yield back domestic.The US is an insider nation with regard to the NAFTA nations, Canada and Mexico, whereas it is an untouchable nation with regard to the coordinates European showcase. The US has distant-coming to special exchanging assentions with Israel, but so has the EU. The nations in Southeast Asia are ordinary low-cost nations that, not at all like Mexico and the nations in Southern Europe, don't have a place to a territorial free-trade region. We in this way need to utilise Southeast Asian nations as a group against which to assess comes about for NAFTA and the EU. Israel will be indicated independently as well.Effective teach secure property rights. The universal economy is getting to be a knowledge-based economy; as a result, mental property rights are getting to be ever more imperative. Their assurance is essential since those who deliver merchandise and administrations can be enough remunerated. The esteem of property rights can rapidly be crushed unless governments uphold rights in this range (Divider et al., 2010). The degree to which outside undertakings can increment ventures in a nation depends on the degree to which that nation has made teach and arrangements for ensuring mental property rights, and a government's adequacy in implementing these rights influences the choices of outside undertakings to enter a host nation, and how they work once there. Frail mental property rights dishearten remote financial specialists, especially in technology-intensive divisions, and energise ventures in ventures that center on dispersion, that's , trading to the host nation, instead of setting up generation plants that might give FDI spillovers; (Rondinelli, 2005).Promoting such offerings within the Joined together States moreover requires another move by firms and brands, that's , numerous analysts and indeed a few professionals respect extravagance shoppers as homogeneous (Dubois 1992). However in Europe, for illustration, unobtrusive social and behavioural assortment and contrasts rise among German, Danish, French, Spanish, and Irish extravagance buyers, likely due to the solid French and Italian impacts that regularly get forced on other European national markets. Extravagance utilisation behaviour can moreover changeessentially between regions, way of life, expendable wage, and over age bunches (Silverstein and Fiske 2003a).The new topic elegant and cool communicates personal taste, separates the respondent from others, and makes a difference illustrate modernity, observing capacities, or victory. Buyers in this way utilise items to send signals around who they are and their objectives. This topic moreover recognises the wants to seem “hip,” see in vogue, and feel special. Unmentionables, design extras, observes, and gems empower the participants, as buyers, to specific their style, information, taste, and values. In spite of the fact that our members don't show up driven basically by a crave for status, nor do they express uncritical captivation with a brand title, they still care almost the messages that merchandise and brands provide approximately their person fashion.The GDP development variable is positive and noteworthy. This suggests that outside speculators are pulled in to have nations with huge markets (market-seeking FDI). This finding is reliable with Bannaga et al. (2013).Trade openness variable is positive and critical. Potential remote speculators are able to gotten to be well educated approximately the nearby conditions of their worldwide exchange accomplices when exchange is freed. Subsequently, outside financial specialists favour contributing in nations with sizable exchange volume. This result is in line with Elfakhani and Matar (2007).Middle easterner economies ought to decrease political hazard by democratising their political frameworks, which may offer assistance settle the locale. They may accomplish legislative soundness by following the equitable change to worldwide community benchmarks in terms of regard for crucial human rights, reinforcing the role of law and advancing national decisions. Middle easterner economies ought to control debasement by supporting the run the show of law and building a successful, unbiased and straightforward legitimate framework. The successful procedure to upgrade the run the show of law requires the foundation of a legal framework in which all people, teach and indeed the government are responsible to laws that are consistently implemented and freely settled. Speculation legislations must secure and ensure ventures within the locale, and ought to be coherent with worldwide guidelines to extend certainty and decrease the risk of apportionment or nationalisation of remote speculations. There's a requirement for the locale to advance law based responsibility by receiving a administration framework based on regulation commitments in giving straightforward data almost open decisions and empowering gracious society to redress or authorise choices that don't follow to the order of law. Constitutions ought to avoid political parties from setting up or getting to be included in military strengths, to avoid the dangers of take-over of a chosen government or of debilitating its capacity to operate viably.",
                "LaDuke’s organisation restricts the proposed course for the modern pipeline, dreading that development and potential spills along the modern course would debilitate the wellbeing of watersheds and Ojibwe lands in Minnesota. “You don’t get to take off a mess in put for the state of Minnesota and tribes, who are exceptionally affected by this since it crosses three reservations, to clean up,” says LaDuke.In Canada, the modern Line 3 has as of now been built. The ancient line will be cleaned out and surrendered in put another year. The country’s National Vitality Board directs pipelines that cross borders and requires companies to yield an arrangement some time recently abandoning a pipeline that incorporates how they’ll set aside reserves to screen the decommissioned pipeline and pay attention of any issues that emerge. It’s a arrangement that’s missing within the US, in spite of the fact that the US Pipeline and Dangerous Materials Security Organisation requires deserted pipelines cleared out in put to be detached, cleansed of unsafe materials, and fixed.An evaluated 140 million individuals misplaced their employments after India bolted down its economy in Walk to halt the flare-up, whereas numerous others saw their pay rates definitely decreased, the Mumbai-based Center for Observing Indian Economy said. As the lockdown was facilitated, numerous went back to work, but more than six million individuals who misplaced occupations haven’t found unused employment.In a June study by the All India Producers Organisation, approximately one-third of little and medium-sized undertakings shown that their businesses were past sparing. The industry bunch said that such a “mass pulverisation of business” was unprecedented.As a run the show, they are created utilising physical strategies. For occurrence, much appreciated to the most modest high-frequency electron developments, the electrical resistance of a wire isn't consistent but instep vacillates marginally in an unpredictable way. Meaning estimations of this foundation commotion can be utilised to produce genuine irregular numbers.Presently, for the primary time, an investigation group driven by Robert Grass, Teacher at the Established of Chemical and Bioengineering, has portrayed a non-physical strategy of producing such numbers: one that employments biochemical signals and actually works in hone. Within the past, the thoughts put forward by other researchers for producing arbitrary numbers by chemical implies tended to be generally hypothetical.It’s all the more vital since the long run for oil and gas companies is more questionable than ever, which makes long-term of their pipelines more questionable, as well. The time will come when the US will got to figure with the destiny of all its pipelines — and when that time comes, the companies that worked them seem now not be around to bargain with them. The COVID-19 widespread alone sent oil and gas companies faltering toward insolvency. Indeed after the economy recoups from the pandemic, there will be more challenges ahead. Worldwide endeavours to handle climate alter, embraced within the point of interest 2015 Paris climate agreement, set countries on a way to cut down nursery gas outflows from burning fossil fills to net zero by 2050. Meaning a future with more renewables and small to no oil and gas.In long haul as unrefined oil request diminishes, we would anticipate to see increasingly pipelines being abandoned,” says Blackburn. “With the oil industry having less and less cash within the future, the hazard gets to be more noteworthy and more prominent that they’ll fair walk absent from their pipelines and take off landowners holding the bag.In any case, an examination appeared that the conveyance of the four building squares A, C, G and T was not totally indeed. Either the complexities of nature or the amalgamation strategy conveyed driven to the bases G and T being coordinates more habitually within the atoms than A and C. Inany case, the researchers were able to adjust this predisposition with a straightforward calculation, subsequently producing idealise irregular numbers.The most point of ETH Teacher Grass and his group was to appear that arbitrary events in chemical response can be misused to produce culminate irregular numbers. Interpreting the finding into a coordinate application was not a prime concern at to begin with. \"Compared with other strategies, be that as it may, our own has the advantage of being able to produce colossal amounts of haphazardness that can be put away in an amazingly little space, a single test tube,\" Grass says. \"Able to examined out the data and reinterpret it in computerised shape at a afterward date. Usually incomprehensible with the past methods.”India’s economy shrank 7.5 percent within the three months that finished in September compared with a year prior, government figures appeared on Friday. The information reflects the extending of India’s severest subsidence since at slightest 1996, when the nation to begin with started distributing its net household item numbers.The unused figures immovably hidden India’s position among the world’s worst-performing major economies, in spite of broad government investing outlined to protect the thousands of little businesses seriously battered by its long, quickly forced lockdown.Material manufacturing plants, calfskin tanneries, brick ovens, foundries and other little ventures shape “part of the country’s social texture, bringing nearby riches and neighborhood employment,” said Venkatachalam Anbumozhi, an financial specialist who centers on south and East Asia. Just many a long time back, India, with a populace of 1.3 billion individuals, was one of the world’s fastest-growing huge economies. It frequently clocked development of 8 percent or more.Global businesses started to warm to the thought of India as a potential substitute to China, both as a put to create merchandise and to offer them. China’s costs are rising, and its exchange war with the Joined together States has complicated doing trade there. The Chinese Communist Party is progressively interfering into commerce things, and nearby Chinese competitors have increased their amusement against universal brands."]
samp_candidate = ["The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively.",
                  "LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands."]

samp_suspect_long = ["The significance of export-platform FDI is reported in a think about by Hanson, Mataloni, and Butcher (2001). Utilising information on the remote operations of US multinationals, they report that in spite of the fact that the normal share of trades in partner deals has remained steady at approximately one-third, there has been a significant increment in Mexico and Canada after the arrangement of NAFTA. Their econometric examination recommends that trade stage FDI is advanced by moo host-country exchange boundaries and disheartened by expansive host-country markets.Within the past decade, the worldwide commercial center has experienced the democratisation of extravagance, driven by a continually developing optimistic customer portion. This prevalent frame of unused extravagance (Silverstein and Fiske 2003a) centers on human association, self- expression, unlimited dissemination, and acknowledgment from others. Unused extravagance, too alluded to as “masstige” (i.e., mass glory; Cornell 2002), challenges routine convictions around extravagance showcasing and branding, counting estimating and cost focuses, brand expansions, buyer request, modernity, showcase steadiness, and the time required to reach the center lesson. The promoting of extravagance has contributed altogether to the democratisation of extravagance, but its impact truly is auxiliary to the impact of changing utilisation designs.The primary case we consider includes symmetric exchange costs on all joins, so that the two firms will each embrace the same number of plants, and either both or not one or the other will have a plant in S and, when they do, the trade designs of those plants will be symmetric. Third-country trade-stage generation emerges when moo generation costs in S lead a firm to utilise S to serve the other N nation, but the investment funds on settled costs from closing the domestic plant don't balanced the costs of shipping components to S and last yield back domestic.The US is an insider nation with regard to the NAFTA nations, Canada and Mexico, whereas it is an untouchable nation with regard to the coordinates European showcase. The US has distant-coming to special exchanging assentions with Israel, but so has the EU. The nations in Southeast Asia are ordinary low-cost nations that, not at all like Mexico and the nations in Southern Europe, don't have a place to a territorial free-trade region. We in this way need to utilise Southeast Asian nations as a group against which to assess comes about for NAFTA and the EU. Israel will be indicated independently as well.Effective teach secure property rights. The universal economy is getting to be a knowledge-based economy; as a result, mental property rights are getting to be ever more imperative. Their assurance is essential since those who deliver merchandise and administrations can be enough remunerated. The esteem of property rights can rapidly be crushed unless governments uphold rights in this range (Divider et al., 2010). The degree to which outside undertakings can increment ventures in a nation depends on the degree to which that nation has made teach and arrangements for ensuring mental property rights, and a government's adequacy in implementing these rights influences the choices of outside undertakings to enter a host nation, and how they work once there. Frail mental property rights dishearten remote financial specialists, especially in technology-intensive divisions, and energise ventures in ventures that center on dispersion, that's , trading to the host nation, instead of setting up generation plants that might give FDI spillovers; (Rondinelli, 2005).Promoting such offerings within the Joined together States moreover requires another move by firms and brands, that's , numerous analysts and indeed a few professionals respect extravagance shoppers as homogeneous (Dubois 1992). However in Europe, for illustration, unobtrusive social and behavioural assortment and contrasts rise among German, Danish, French, Spanish, and Irish extravagance buyers, likely due to the solid French and Italian impacts that regularly get forced on other European national markets. Extravagance utilisation behaviour can moreover changeessentially between regions, way of life, expendable wage, and over age bunches (Silverstein and Fiske 2003a).The new topic elegant and cool communicates personal taste, separates the respondent from others, and makes a difference illustrate modernity, observing capacities, or victory. Buyers in this way utilise items to send signals around who they are and their objectives. This topic moreover recognises the wants to seem “hip,” see in vogue, and feel special. Unmentionables, design extras, observes, and gems empower the participants, as buyers, to specific their style, information, taste, and values. In spite of the fact that our members don't show up driven basically by a crave for status, nor do they express uncritical captivation with a brand title, they still care almost the messages that merchandise and brands provide approximately their person fashion.The GDP development variable is positive and noteworthy. This suggests that outside speculators are pulled in to have nations with huge markets (market-seeking FDI). This finding is reliable with Bannaga et al. (2013).Trade openness variable is positive and critical. Potential remote speculators are able to gotten to be well educated approximately the nearby conditions of their worldwide exchange accomplices when exchange is freed. Subsequently, outside financial specialists favour contributing in nations with sizable exchange volume. This result is in line with Elfakhani and Matar (2007).Middle easterner economies ought to decrease political hazard by democratising their political frameworks, which may offer assistance settle the locale. They may accomplish legislative soundness by following the equitable change to worldwide community benchmarks in terms of regard for crucial human rights, reinforcing the role of law and advancing national decisions. Middle easterner economies ought to control debasement by supporting the run the show of law and building a successful, unbiased and straightforward legitimate framework. The successful procedure to upgrade the run the show of law requires the foundation of a legal framework in which all people, teach and indeed the government are responsible to laws that are consistently implemented and freely settled. Speculation legislations must secure and ensure ventures within the locale, and ought to be coherent with worldwide guidelines to extend certainty and decrease the risk of apportionment or nationalisation of remote speculations. There's a requirement for the locale to advance law based responsibility by receiving a administration framework based on regulation commitments in giving straightforward data almost open decisions and empowering gracious society to redress or authorise choices that don't follow to the order of law. Constitutions ought to avoid political parties from setting up or getting to be included in military strengths, to avoid the dangers of take-over of a chosen government or of debilitating its capacity to operate viably. LaDuke’s organisation restricts the proposed course for the modern pipeline, dreading that development and potential spills along the modern course would debilitate the wellbeing of watersheds and Ojibwe lands in Minnesota. “You don’t get to take off a mess in put for the state of Minnesota and tribes, who are exceptionally affected by this since it crosses three reservations, to clean up,” says LaDuke.In Canada, the modern Line 3 has as of now been built. The ancient line will be cleaned out and surrendered in put another year. The country’s National Vitality Board directs pipelines that cross borders and requires companies to yield an arrangement some time recently abandoning a pipeline that incorporates how they’ll set aside reserves to screen the decommissioned pipeline and pay attention of any issues that emerge. It’s a arrangement that’s missing within the US, in spite of the fact that the US Pipeline and Dangerous Materials Security Organisation requires deserted pipelines cleared out in put to be detached, cleansed of unsafe materials, and fixed.An evaluated 140 million individuals misplaced their employments after India bolted down its economy in Walk to halt the flare-up, whereas numerous others saw their pay rates definitely decreased, the Mumbai-based Center for Observing Indian Economy said. As the lockdown was facilitated, numerous went back to work, but more than six million individuals who misplaced occupations haven’t found unused employment.In a June study by the All India Producers Organisation, approximately one-third of little and medium-sized undertakings shown that their businesses were past sparing. The industry bunch said that such a “mass pulverisation of business” was unprecedented.As a run the show, they are created utilising physical strategies. For occurrence, much appreciated to the most modest high-frequency electron developments, the electrical resistance of a wire isn't consistent but instep vacillates marginally in an unpredictable way. Meaning estimations of this foundation commotion can be utilised to produce genuine irregular numbers.Presently, for the primary time, an investigation group driven by Robert Grass, Teacher at the Established of Chemical and Bioengineering, has portrayed a non-physical strategy of producing such numbers: one that employments biochemical signals and actually works in hone. Within the past, the thoughts put forward by other researchers for producing arbitrary numbers by chemical implies tended to be generally hypothetical.It’s all the more vital since the long run for oil and gas companies is more questionable than ever, which makes long-term of their pipelines more questionable, as well. The time will come when the US will got to figure with the destiny of all its pipelines — and when that time comes, the companies that worked them seem now not be around to bargain with them. The COVID-19 widespread alone sent oil and gas companies faltering toward insolvency. Indeed after the economy recoups from the pandemic, there will be more challenges ahead. Worldwide endeavours to handle climate alter, embraced within the point of interest 2015 Paris climate agreement, set countries on a way to cut down nursery gas outflows from burning fossil fills to net zero by 2050. Meaning a future with more renewables and small to no oil and gas.In long haul as unrefined oil request diminishes, we would anticipate to see increasingly pipelines being abandoned,” says Blackburn. “With the oil industry having less and less cash within the future, the hazard gets to be more noteworthy and more prominent that they’ll fair walk absent from their pipelines and take off landowners holding the bag.In any case, an examination appeared that the conveyance of the four building squares A, C, G and T was not totally indeed. Either the complexities of nature or the amalgamation strategy conveyed driven to the bases G and T being coordinates more habitually within the atoms than A and C. Inany case, the researchers were able to adjust this predisposition with a straightforward calculation, subsequently producing idealise irregular numbers.The most point of ETH Teacher Grass and his group was to appear that arbitrary events in chemical response can be misused to produce culminate irregular numbers. Interpreting the finding into a coordinate application was not a prime concern at to begin with. \"Compared with other strategies, be that as it may, our own has the advantage of being able to produce colossal amounts of haphazardness that can be put away in an amazingly little space, a single test tube,\" Grass says. \"Able to examined out the data and reinterpret it in computerised shape at a afterward date. Usually incomprehensible with the past methods.”India’s economy shrank 7.5 percent within the three months that finished in September compared with a year prior, government figures appeared on Friday. The information reflects the extending of India’s severest subsidence since at slightest 1996, when the nation to begin with started distributing its net household item numbers.The unused figures immovably hidden India’s position among the world’s worst-performing major economies, in spite of broad government investing outlined to protect the thousands of little businesses seriously battered by its long, quickly forced lockdown.Material manufacturing plants, calfskin tanneries, brick ovens, foundries and other little ventures shape “part of the country’s social texture, bringing nearby riches and neighborhood employment,” said Venkatachalam Anbumozhi, an financial specialist who centers on south and East Asia. Just many a long time back, India, with a populace of 1.3 billion individuals, was one of the world’s fastest-growing huge economies. It frequently clocked development of 8 percent or more.Global businesses started to warm to the thought of India as a potential substitute to China, both as a put to create merchandise and to offer them. China’s costs are rising, and its exchange war with the Joined together States has complicated doing trade there. The Chinese Communist Party is progressively interfering into commerce things, and nearby Chinese competitors have increased their amusement against universal brands. The significance of export-platform FDI is reported in a think about by Hanson, Mataloni, and Butcher (2001). Utilising information on the remote operations of US multinationals, they report that in spite of the fact that the normal share of trades in partner deals has remained steady at approximately one-third, there has been a significant increment in Mexico and Canada after the arrangement of NAFTA. Their econometric examination recommends that trade stage FDI is advanced by moo host-country exchange boundaries and disheartened by expansive host-country markets.Within the past decade, the worldwide commercial center has experienced the democratisation of extravagance, driven by a continually developing optimistic customer portion. This prevalent frame of unused extravagance (Silverstein and Fiske 2003a) centers on human association, self- expression, unlimited dissemination, and acknowledgment from others. Unused extravagance, too alluded to as “masstige” (i.e., mass glory; Cornell 2002), challenges routine convictions around extravagance showcasing and branding, counting estimating and cost focuses, brand expansions, buyer request, modernity, showcase steadiness, and the time required to reach the center lesson. The promoting of extravagance has contributed altogether to the democratisation of extravagance, but its impact truly is auxiliary to the impact of changing utilisation designs.The primary case we consider includes symmetric exchange costs on all joins, so that the two firms will each embrace the same number of plants, and either both or not one or the other will have a plant in S and, when they do, the trade designs of those plants will be symmetric. Third-country trade-stage generation emerges when moo generation costs in S lead a firm to utilise S to serve the other N nation, but the investment funds on settled costs from closing the domestic plant don't balanced the costs of shipping components to S and last yield back domestic.The US is an insider nation with regard to the NAFTA nations, Canada and Mexico, whereas it is an untouchable nation with regard to the coordinates European showcase. The US has distant-coming to special exchanging assentions with Israel, but so has the EU. The nations in Southeast Asia are ordinary low-cost nations that, not at all like Mexico and the nations in Southern Europe, don't have a place to a territorial free-trade region. We in this way need to utilise Southeast Asian nations as a group against which to assess comes about for NAFTA and the EU. Israel will be indicated independently as well.Effective teach secure property rights. The universal economy is getting to be a knowledge-based economy; as a result, mental property rights are getting to be ever more imperative. Their assurance is essential since those who deliver merchandise and administrations can be enough remunerated. The esteem of property rights can rapidly be crushed unless governments uphold rights in this range (Divider et al., 2010). The degree to which outside undertakings can increment ventures in a nation depends on the degree to which that nation has made teach and arrangements for ensuring mental property rights, and a government's adequacy in implementing these rights influences the choices of outside undertakings to enter a host nation, and how they work once there. Frail mental property rights dishearten remote financial specialists, especially in technology-intensive divisions, and energise ventures in ventures that center on dispersion, that's , trading to the host nation, instead of setting up generation plants that might give FDI spillovers; (Rondinelli, 2005).Promoting such offerings within the Joined together States moreover requires another move by firms and brands, that's , numerous analysts and indeed a few professionals respect extravagance shoppers as homogeneous (Dubois 1992). However in Europe, for illustration, unobtrusive social and behavioural assortment and contrasts rise among German, Danish, French, Spanish, and Irish extravagance buyers, likely due to the solid French and Italian impacts that regularly get forced on other European national markets. Extravagance utilisation behaviour can moreover changeessentially between regions, way of life, expendable wage, and over age bunches (Silverstein and Fiske 2003a).The new topic elegant and cool communicates personal taste, separates the respondent from others, and makes a difference illustrate modernity, observing capacities, or victory. Buyers in this way utilise items to send signals around who they are and their objectives. This topic moreover recognises the wants to seem “hip,” see in vogue, and feel special. Unmentionables, design extras, observes, and gems empower the participants, as buyers, to specific their style, information, taste, and values. In spite of the fact that our members don't show up driven basically by a crave for status, nor do they express uncritical captivation with a brand title, they still care almost the messages that merchandise and brands provide approximately their person fashion.The GDP development variable is positive and noteworthy. This suggests that outside speculators are pulled in to have nations with huge markets (market-seeking FDI). This finding is reliable with Bannaga et al. (2013).Trade openness variable is positive and critical. Potential remote speculators are able to gotten to be well educated approximately the nearby conditions of their worldwide exchange accomplices when exchange is freed. Subsequently, outside financial specialists favour contributing in nations with sizable exchange volume. This result is in line with Elfakhani and Matar (2007).Middle easterner economies ought to decrease political hazard by democratising their political frameworks, which may offer assistance settle the locale. They may accomplish legislative soundness by following the equitable change to worldwide community benchmarks in terms of regard for crucial human rights, reinforcing the role of law and advancing national decisions. Middle easterner economies ought to control debasement by supporting the run the show of law and building a successful, unbiased and straightforward legitimate framework. The successful procedure to upgrade the run the show of law requires the foundation of a legal framework in which all people, teach and indeed the government are responsible to laws that are consistently implemented and freely settled. Speculation legislations must secure and ensure ventures within the locale, and ought to be coherent with worldwide guidelines to extend certainty and decrease the risk of apportionment or nationalisation of remote speculations. There's a requirement for the locale to advance law based responsibility by receiving a administration framework based on regulation commitments in giving straightforward data almost open decisions and empowering gracious society to redress or authorise choices that don't follow to the order of law. Constitutions ought to avoid political parties from setting up or getting to be included in military strengths, to avoid the dangers of take-over of a chosen government or of debilitating its capacity to operate viably. LaDuke’s organisation restricts the proposed course for the modern pipeline, dreading that development and potential spills along the modern course would debilitate the wellbeing of watersheds and Ojibwe lands in Minnesota. “You don’t get to take off a mess in put for the state of Minnesota and tribes, who are exceptionally affected by this since it crosses three reservations, to clean up,” says LaDuke.In Canada, the modern Line 3 has as of now been built. The ancient line will be cleaned out and surrendered in put another year. The country’s National Vitality Board directs pipelines that cross borders and requires companies to yield an arrangement some time recently abandoning a pipeline that incorporates how they’ll set aside reserves to screen the decommissioned pipeline and pay attention of any issues that emerge. It’s a arrangement that’s missing within the US, in spite of the fact that the US Pipeline and Dangerous Materials Security Organisation requires deserted pipelines cleared out in put to be detached, cleansed of unsafe materials, and fixed.An evaluated 140 million individuals misplaced their employments after India bolted down its economy in Walk to halt the flare-up, whereas numerous others saw their pay rates definitely decreased, the Mumbai-based Center for Observing Indian Economy said. As the lockdown was facilitated, numerous went back to work, but more than six million individuals who misplaced occupations haven’t found unused employment.In a June study by the All India Producers Organisation, approximately one-third of little and medium-sized undertakings shown that their businesses were past sparing. The industry bunch said that such a “mass pulverisation of business” was unprecedented.As a run the show, they are created utilising physical strategies. For occurrence, much appreciated to the most modest high-frequency electron developments, the electrical resistance of a wire isn't consistent but instep vacillates marginally in an unpredictable way. Meaning estimations of this foundation commotion can be utilised to produce genuine irregular numbers.Presently, for the primary time, an investigation group driven by Robert Grass, Teacher at the Established of Chemical and Bioengineering, has portrayed a non-physical strategy of producing such numbers: one that employments biochemical signals and actually works in hone. Within the past, the thoughts put forward by other researchers for producing arbitrary numbers by chemical implies tended to be generally hypothetical.It’s all the more vital since the long run for oil and gas companies is more questionable than ever, which makes long-term of their pipelines more questionable, as well. The time will come when the US will got to figure with the destiny of all its pipelines — and when that time comes, the companies that worked them seem now not be around to bargain with them. The COVID-19 widespread alone sent oil and gas companies faltering toward insolvency. Indeed after the economy recoups from the pandemic, there will be more challenges ahead. Worldwide endeavours to handle climate alter, embraced within the point of interest 2015 Paris climate agreement, set countries on a way to cut down nursery gas outflows from burning fossil fills to net zero by 2050. Meaning a future with more renewables and small to no oil and gas.In long haul as unrefined oil request diminishes, we would anticipate to see increasingly pipelines being abandoned,” says Blackburn. “With the oil industry having less and less cash within the future, the hazard gets to be more noteworthy and more prominent that they’ll fair walk absent from their pipelines and take off landowners holding the bag.In any case, an examination appeared that the conveyance of the four building squares A, C, G and T was not totally indeed. Either the complexities of nature or the amalgamation strategy conveyed driven to the bases G and T being coordinates more habitually within the atoms than A and C. Inany case, the researchers were able to adjust this predisposition with a straightforward calculation, subsequently producing idealise irregular numbers.The most point of ETH Teacher Grass and his group was to appear that arbitrary events in chemical response can be misused to produce culminate irregular numbers. Interpreting the finding into a coordinate application was not a prime concern at to begin with. \"Compared with other strategies, be that as it may, our own has the advantage of being able to produce colossal amounts of haphazardness that can be put away in an amazingly little space, a single test tube,\" Grass says. \"Able to examined out the data and reinterpret it in computerised shape at a afterward date. Usually incomprehensible with the past methods.”India’s economy shrank 7.5 percent within the three months that finished in September compared with a year prior, government figures appeared on Friday. The information reflects the extending of India’s severest subsidence since at slightest 1996, when the nation to begin with started distributing its net household item numbers.The unused figures immovably hidden India’s position among the world’s worst-performing major economies, in spite of broad government investing outlined to protect the thousands of little businesses seriously battered by its long, quickly forced lockdown.Material manufacturing plants, calfskin tanneries, brick ovens, foundries and other little ventures shape “part of the country’s social texture, bringing nearby riches and neighborhood employment,” said Venkatachalam Anbumozhi, an financial specialist who centers on south and East Asia. Just many a long time back, India, with a populace of 1.3 billion individuals, was one of the world’s fastest-growing huge economies. It frequently clocked development of 8 percent or more.Global businesses started to warm to the thought of India as a potential substitute to China, both as a put to create merchandise and to offer them. China’s costs are rising, and its exchange war with the Joined together States has complicated doing trade there. The Chinese Communist Party is progressively interfering into commerce things, and nearby Chinese competitors have increased their amusement against universal brands. The significance of export-platform FDI is reported in a think about by Hanson, Mataloni, and Butcher (2001). Utilising information on the remote operations of US multinationals, they report that in spite of the fact that the normal share of trades in partner deals has remained steady at approximately one-third, there has been a significant increment in Mexico and Canada after the arrangement of NAFTA. Their econometric examination recommends that trade stage FDI is advanced by moo host-country exchange boundaries and disheartened by expansive host-country markets.Within the past decade, the worldwide commercial center has experienced the democratisation of extravagance, driven by a continually developing optimistic customer portion. This prevalent frame of unused extravagance (Silverstein and Fiske 2003a) centers on human association, self- expression, unlimited dissemination, and acknowledgment from others. Unused extravagance, too alluded to as “masstige” (i.e., mass glory; Cornell 2002), challenges routine convictions around extravagance showcasing and branding, counting estimating and cost focuses, brand expansions, buyer request, modernity, showcase steadiness, and the time required to reach the center lesson. The promoting of extravagance has contributed altogether to the democratisation of extravagance, but its impact truly is auxiliary to the impact of changing utilisation designs.The primary case we consider includes symmetric exchange costs on all joins, so that the two firms will each embrace the same number of plants, and either both or not one or the other will have a plant in S and, when they do, the trade designs of those plants will be symmetric. Third-country trade-stage generation emerges when moo generation costs in S lead a firm to utilise S to serve the other N nation, but the investment funds on settled costs from closing the domestic plant don't balanced the costs of shipping components to S and last yield back domestic.The US is an insider nation with regard to the NAFTA nations, Canada and Mexico, whereas it is an untouchable nation with regard to the coordinates European showcase. The US has distant-coming to special exchanging assentions with Israel, but so has the EU. The nations in Southeast Asia are ordinary low-cost nations that, not at all like Mexico and the nations in Southern Europe, don't have a place to a territorial free-trade region. We in this way need to utilise Southeast Asian nations as a group against which to assess comes about for NAFTA and the EU. Israel will be indicated independently as well.Effective teach secure property rights. The universal economy is getting to be a knowledge-based economy; as a result, mental property rights are getting to be ever more imperative. Their assurance is essential since those who deliver merchandise and administrations can be enough remunerated. The esteem of property rights can rapidly be crushed unless governments uphold rights in this range (Divider et al., 2010). The degree to which outside undertakings can increment ventures in a nation depends on the degree to which that nation has made teach and arrangements for ensuring mental property rights, and a government's adequacy in implementing these rights influences the choices of outside undertakings to enter a host nation, and how they work once there. Frail mental property rights dishearten remote financial specialists, especially in technology-intensive divisions, and energise ventures in ventures that center on dispersion, that's , trading to the host nation, instead of setting up generation plants that might give FDI spillovers; (Rondinelli, 2005).Promoting such offerings within the Joined together States moreover requires another move by firms and brands, that's , numerous analysts and indeed a few professionals respect extravagance shoppers as homogeneous (Dubois 1992). However in Europe, for illustration, unobtrusive social and behavioural assortment and contrasts rise among German, Danish, French, Spanish, and Irish extravagance buyers, likely due to the solid French and Italian impacts that regularly get forced on other European national markets. Extravagance utilisation behaviour can moreover changeessentially between regions, way of life, expendable wage, and over age bunches (Silverstein and Fiske 2003a).The new topic elegant and cool communicates personal taste, separates the respondent from others, and makes a difference illustrate modernity, observing capacities, or victory. Buyers in this way utilise items to send signals around who they are and their objectives. This topic moreover recognises the wants to seem “hip,” see in vogue, and feel special. Unmentionables, design extras, observes, and gems empower the participants, as buyers, to specific their style, information, taste, and values. In spite of the fact that our members don't show up driven basically by a crave for status, nor do they express uncritical captivation with a brand title, they still care almost the messages that merchandise and brands provide approximately their person fashion.The GDP development variable is positive and noteworthy. This suggests that outside speculators are pulled in to have nations with huge markets (market-seeking FDI). This finding is reliable with Bannaga et al. (2013).Trade openness variable is positive and critical. Potential remote speculators are able to gotten to be well educated approximately the nearby conditions of their worldwide exchange accomplices when exchange is freed. Subsequently, outside financial specialists favour contributing in nations with sizable exchange volume. This result is in line with Elfakhani and Matar (2007).Middle easterner economies ought to decrease political hazard by democratising their political frameworks, which may offer assistance settle the locale. They may accomplish legislative soundness by following the equitable change to worldwide community benchmarks in terms of regard for crucial human rights, reinforcing the role of law and advancing national decisions. Middle easterner economies ought to control debasement by supporting the run the show of law and building a successful, unbiased and straightforward legitimate framework. The successful procedure to upgrade the run the show of law requires the foundation of a legal framework in which all people, teach and indeed the government are responsible to laws that are consistently implemented and freely settled. Speculation legislations must secure and ensure ventures within the locale, and ought to be coherent with worldwide guidelines to extend certainty and decrease the risk of apportionment or nationalisation of remote speculations. There's a requirement for the locale to advance law based responsibility by receiving a administration framework based on regulation commitments in giving straightforward data almost open decisions and empowering gracious society to redress or authorise choices that don't follow to the order of law. Constitutions ought to avoid political parties from setting up or getting to be included in military strengths, to avoid the dangers of take-over of a chosen government or of debilitating its capacity to operate viably. LaDuke’s organisation restricts the proposed course for the modern pipeline, dreading that development and potential spills along the modern course would debilitate the wellbeing of watersheds and Ojibwe lands in Minnesota. “You don’t get to take off a mess in put for the state of Minnesota and tribes, who are exceptionally affected by this since it crosses three reservations, to clean up,” says LaDuke.In Canada, the modern Line 3 has as of now been built. The ancient line will be cleaned out and surrendered in put another year. The country’s National Vitality Board directs pipelines that cross borders and requires companies to yield an arrangement some time recently abandoning a pipeline that incorporates how they’ll set aside reserves to screen the decommissioned pipeline and pay attention of any issues that emerge. It’s a arrangement that’s missing within the US, in spite of the fact that the US Pipeline and Dangerous Materials Security Organisation requires deserted pipelines cleared out in put to be detached, cleansed of unsafe materials, and fixed.An evaluated 140 million individuals misplaced their employments after India bolted down its economy in Walk to halt the flare-up, whereas numerous others saw their pay rates definitely decreased, the Mumbai-based Center for Observing Indian Economy said. As the lockdown was facilitated, numerous went back to work, but more than six million individuals who misplaced occupations haven’t found unused employment.In a June study by the All India Producers Organisation, approximately one-third of little and medium-sized undertakings shown that their businesses were past sparing. The industry bunch said that such a “mass pulverisation of business” was unprecedented.As a run the show, they are created utilising physical strategies. For occurrence, much appreciated to the most modest high-frequency electron developments, the electrical resistance of a wire isn't consistent but instep vacillates marginally in an unpredictable way. Meaning estimations of this foundation commotion can be utilised to produce genuine irregular numbers.Presently, for the primary time, an investigation group driven by Robert Grass, Teacher at the Established of Chemical and Bioengineering, has portrayed a non-physical strategy of producing such numbers: one that employments biochemical signals and actually works in hone. Within the past, the thoughts put forward by other researchers for producing arbitrary numbers by chemical implies tended to be generally hypothetical.It’s all the more vital since the long run for oil and gas companies is more questionable than ever, which makes long-term of their pipelines more questionable, as well. The time will come when the US will got to figure with the destiny of all its pipelines — and when that time comes, the companies that worked them seem now not be around to bargain with them. The COVID-19 widespread alone sent oil and gas companies faltering toward insolvency. Indeed after the economy recoups from the pandemic, there will be more challenges ahead. Worldwide endeavours to handle climate alter, embraced within the point of interest 2015 Paris climate agreement, set countries on a way to cut down nursery gas outflows from burning fossil fills to net zero by 2050. Meaning a future with more renewables and small to no oil and gas.In long haul as unrefined oil request diminishes, we would anticipate to see increasingly pipelines being abandoned,” says Blackburn. “With the oil industry having less and less cash within the future, the hazard gets to be more noteworthy and more prominent that they’ll fair walk absent from their pipelines and take off landowners holding the bag.In any case, an examination appeared that the conveyance of the four building squares A, C, G and T was not totally indeed. Either the complexities of nature or the amalgamation strategy conveyed driven to the bases G and T being coordinates more habitually within the atoms than A and C. Inany case, the researchers were able to adjust this predisposition with a straightforward calculation, subsequently producing idealise irregular numbers.The most point of ETH Teacher Grass and his group was to appear that arbitrary events in chemical response can be misused to produce culminate irregular numbers. Interpreting the finding into a coordinate application was not a prime concern at to begin with. \"Compared with other strategies, be that as it may, our own has the advantage of being able to produce colossal amounts of haphazardness that can be put away in an amazingly little space, a single test tube,\" Grass says. \"Able to examined out the data and reinterpret it in computerised shape at a afterward date. Usually incomprehensible with the past methods.”India’s economy shrank 7.5 percent within the three months that finished in September compared with a year prior, government figures appeared on Friday. The information reflects the extending of India’s severest subsidence since at slightest 1996, when the nation to begin with started distributing its net household item numbers.The unused figures immovably hidden India’s position among the world’s worst-performing major economies, in spite of broad government investing outlined to protect the thousands of little businesses seriously battered by its long, quickly forced lockdown.Material manufacturing plants, calfskin tanneries, brick ovens, foundries and other little ventures shape “part of the country’s social texture, bringing nearby riches and neighborhood employment,” said Venkatachalam Anbumozhi, an financial specialist who centers on south and East Asia. Just many a long time back, India, with a populace of 1.3 billion individuals, was one of the world’s fastest-growing huge economies. It frequently clocked development of 8 percent or more.Global businesses started to warm to the thought of India as a potential substitute to China, both as a put to create merchandise and to offer them. China’s costs are rising, and its exchange war with the Joined together States has complicated doing trade there. The Chinese Communist Party is progressively interfering into commerce things, and nearby Chinese competitors have increased their amusement against universal brands. The significance of export-platform FDI is reported in a think about by Hanson, Mataloni, and Butcher (2001). Utilising information on the remote operations of US multinationals, they report that in spite of the fact that the normal share of trades in partner deals has remained steady at approximately one-third, there has been a significant increment in Mexico and Canada after the arrangement of NAFTA. Their econometric examination recommends that trade stage FDI is advanced by moo host-country exchange boundaries and disheartened by expansive host-country markets.Within the past decade, the worldwide commercial center has experienced the democratisation of extravagance, driven by a continually developing optimistic customer portion. This prevalent frame of unused extravagance (Silverstein and Fiske 2003a) centers on human association, self- expression, unlimited dissemination, and acknowledgment from others. Unused extravagance, too alluded to as “masstige” (i.e., mass glory; Cornell 2002), challenges routine convictions around extravagance showcasing and branding, counting estimating and cost focuses, brand expansions, buyer request, modernity, showcase steadiness, and the time required to reach the center lesson. The promoting of extravagance has contributed altogether to the democratisation of extravagance, but its impact truly is auxiliary to the impact of changing utilisation designs.The primary case we consider includes symmetric exchange costs on all joins, so that the two firms will each embrace the same number of plants, and either both or not one or the other will have a plant in S and, when they do, the trade designs of those plants will be symmetric. Third-country trade-stage generation emerges when moo generation costs in S lead a firm to utilise S to serve the other N nation, but the investment funds on settled costs from closing the domestic plant don't balanced the costs of shipping components to S and last yield back domestic.The US is an insider nation with regard to the NAFTA nations, Canada and Mexico, whereas it is an untouchable nation with regard to the coordinates European showcase. The US has distant-coming to special exchanging assentions with Israel, but so has the EU. The nations in Southeast Asia are ordinary low-cost nations that, not at all like Mexico and the nations in Southern Europe, don't have a place to a territorial free-trade region. We in this way need to utilise Southeast Asian nations as a group against which to assess comes about for NAFTA and the EU. Israel will be indicated independently as well.Effective teach secure property rights. The universal economy is getting to be a knowledge-based economy; as a result, mental property rights are getting to be ever more imperative. Their assurance is essential since those who deliver merchandise and administrations can be enough remunerated. The esteem of property rights can rapidly be crushed unless governments uphold rights in this range (Divider et al., 2010). The degree to which outside undertakings can increment ventures in a nation depends on the degree to which that nation has made teach and arrangements for ensuring mental property rights, and a government's adequacy in implementing these rights influences the choices of outside undertakings to enter a host nation, and how they work once there. Frail mental property rights dishearten remote financial specialists, especially in technology-intensive divisions, and energise ventures in ventures that center on dispersion, that's , trading to the host nation, instead of setting up generation plants that might give FDI spillovers; (Rondinelli, 2005).Promoting such offerings within the Joined together States moreover requires another move by firms and brands, that's , numerous analysts and indeed a few professionals respect extravagance shoppers as homogeneous (Dubois 1992). However in Europe, for illustration, unobtrusive social and behavioural assortment and contrasts rise among German, Danish, French, Spanish, and Irish extravagance buyers, likely due to the solid French and Italian impacts that regularly get forced on other European national markets. Extravagance utilisation behaviour can moreover changeessentially between regions, way of life, expendable wage, and over age bunches (Silverstein and Fiske 2003a).The new topic elegant and cool communicates personal taste, separates the respondent from others, and makes a difference illustrate modernity, observing capacities, or victory. Buyers in this way utilise items to send signals around who they are and their objectives. This topic moreover recognises the wants to seem “hip,” see in vogue, and feel special. Unmentionables, design extras, observes, and gems empower the participants, as buyers, to specific their style, information, taste, and values. In spite of the fact that our members don't show up driven basically by a crave for status, nor do they express uncritical captivation with a brand title, they still care almost the messages that merchandise and brands provide approximately their person fashion.The GDP development variable is positive and noteworthy. This suggests that outside speculators are pulled in to have nations with huge markets (market-seeking FDI). This finding is reliable with Bannaga et al. (2013).Trade openness variable is positive and critical. Potential remote speculators are able to gotten to be well educated approximately the nearby conditions of their worldwide exchange accomplices when exchange is freed. Subsequently, outside financial specialists favour contributing in nations with sizable exchange volume. This result is in line with Elfakhani and Matar (2007).Middle easterner economies ought to decrease political hazard by democratising their political frameworks, which may offer assistance settle the locale. They may accomplish legislative soundness by following the equitable change to worldwide community benchmarks in terms of regard for crucial human rights, reinforcing the role of law and advancing national decisions. Middle easterner economies ought to control debasement by supporting the run the show of law and building a successful, unbiased and straightforward legitimate framework. The successful procedure to upgrade the run the show of law requires the foundation of a legal framework in which all people, teach and indeed the government are responsible to laws that are consistently implemented and freely settled. Speculation legislations must secure and ensure ventures within the locale, and ought to be coherent with worldwide guidelines to extend certainty and decrease the risk of apportionment or nationalisation of remote speculations. There's a requirement for the locale to advance law based responsibility by receiving a administration framework based on regulation commitments in giving straightforward data almost open decisions and empowering gracious society to redress or authorise choices that don't follow to the order of law. Constitutions ought to avoid political parties from setting up or getting to be included in military strengths, to avoid the dangers of take-over of a chosen government or of debilitating its capacity to operate viably. LaDuke’s organisation restricts the proposed course for the modern pipeline, dreading that development and potential spills along the modern course would debilitate the wellbeing of watersheds and Ojibwe lands in Minnesota. “You don’t get to take off a mess in put for the state of Minnesota and tribes, who are exceptionally affected by this since it crosses three reservations, to clean up,” says LaDuke.In Canada, the modern Line 3 has as of now been built. The ancient line will be cleaned out and surrendered in put another year. The country’s National Vitality Board directs pipelines that cross borders and requires companies to yield an arrangement some time recently abandoning a pipeline that incorporates how they’ll set aside reserves to screen the decommissioned pipeline and pay attention of any issues that emerge. It’s a arrangement that’s missing within the US, in spite of the fact that the US Pipeline and Dangerous Materials Security Organisation requires deserted pipelines cleared out in put to be detached, cleansed of unsafe materials, and fixed.An evaluated 140 million individuals misplaced their employments after India bolted down its economy in Walk to halt the flare-up, whereas numerous others saw their pay rates definitely decreased, the Mumbai-based Center for Observing Indian Economy said. As the lockdown was facilitated, numerous went back to work, but more than six million individuals who misplaced occupations haven’t found unused employment.In a June study by the All India Producers Organisation, approximately one-third of little and medium-sized undertakings shown that their businesses were past sparing. The industry bunch said that such a “mass pulverisation of business” was unprecedented.As a run the show, they are created utilising physical strategies. For occurrence, much appreciated to the most modest high-frequency electron developments, the electrical resistance of a wire isn't consistent but instep vacillates marginally in an unpredictable way. Meaning estimations of this foundation commotion can be utilised to produce genuine irregular numbers.Presently, for the primary time, an investigation group driven by Robert Grass, Teacher at the Established of Chemical and Bioengineering, has portrayed a non-physical strategy of producing such numbers: one that employments biochemical signals and actually works in hone. Within the past, the thoughts put forward by other researchers for producing arbitrary numbers by chemical implies tended to be generally hypothetical.It’s all the more vital since the long run for oil and gas companies is more questionable than ever, which makes long-term of their pipelines more questionable, as well. The time will come when the US will got to figure with the destiny of all its pipelines — and when that time comes, the companies that worked them seem now not be around to bargain with them. The COVID-19 widespread alone sent oil and gas companies faltering toward insolvency. Indeed after the economy recoups from the pandemic, there will be more challenges ahead. Worldwide endeavours to handle climate alter, embraced within the point of interest 2015 Paris climate agreement, set countries on a way to cut down nursery gas outflows from burning fossil fills to net zero by 2050. Meaning a future with more renewables and small to no oil and gas.In long haul as unrefined oil request diminishes, we would anticipate to see increasingly pipelines being abandoned,” says Blackburn. “With the oil industry having less and less cash within the future, the hazard gets to be more noteworthy and more prominent that they’ll fair walk absent from their pipelines and take off landowners holding the bag.In any case, an examination appeared that the conveyance of the four building squares A, C, G and T was not totally indeed. Either the complexities of nature or the amalgamation strategy conveyed driven to the bases G and T being coordinates more habitually within the atoms than A and C. Inany case, the researchers were able to adjust this predisposition with a straightforward calculation, subsequently producing idealise irregular numbers.The most point of ETH Teacher Grass and his group was to appear that arbitrary events in chemical response can be misused to produce culminate irregular numbers. Interpreting the finding into a coordinate application was not a prime concern at to begin with. \"Compared with other strategies, be that as it may, our own has the advantage of being able to produce colossal amounts of haphazardness that can be put away in an amazingly little space, a single test tube,\" Grass says. \"Able to examined out the data and reinterpret it in computerised shape at a afterward date. Usually incomprehensible with the past methods.”India’s economy shrank 7.5 percent within the three months that finished in September compared with a year prior, government figures appeared on Friday. The information reflects the extending of India’s severest subsidence since at slightest 1996, when the nation to begin with started distributing its net household item numbers.The unused figures immovably hidden India’s position among the world’s worst-performing major economies, in spite of broad government investing outlined to protect the thousands of little businesses seriously battered by its long, quickly forced lockdown.Material manufacturing plants, calfskin tanneries, brick ovens, foundries and other little ventures shape “part of the country’s social texture, bringing nearby riches and neighborhood employment,” said Venkatachalam Anbumozhi, an financial specialist who centers on south and East Asia. Just many a long time back, India, with a populace of 1.3 billion individuals, was one of the world’s fastest-growing huge economies. It frequently clocked development of 8 percent or more.Global businesses started to warm to the thought of India as a potential substitute to China, both as a put to create merchandise and to offer them. China’s costs are rising, and its exchange war with the Joined together States has complicated doing trade there. The Chinese Communist Party is progressively interfering into commerce things, and nearby Chinese competitors have increased their amusement against universal brands. The significance of export-platform FDI is reported in a think about by Hanson, Mataloni, and Butcher (2001). Utilising information on the remote operations of US multinationals, they report that in spite of the fact that the normal share of trades in partner deals has remained steady at approximately one-third, there has been a significant increment in Mexico and Canada after the arrangement of NAFTA. Their econometric examination recommends that trade stage FDI is advanced by moo host-country exchange boundaries and disheartened by expansive host-country markets.Within the past decade, the worldwide commercial center has experienced the democratisation of extravagance, driven by a continually developing optimistic customer portion. This prevalent frame of unused extravagance (Silverstein and Fiske 2003a) centers on human association, self- expression, unlimited dissemination, and acknowledgment from others. Unused extravagance, too alluded to as “masstige” (i.e., mass glory; Cornell 2002), challenges routine convictions around extravagance showcasing and branding, counting estimating and cost focuses, brand expansions, buyer request, modernity, showcase steadiness, and the time required to reach the center lesson. The promoting of extravagance has contributed altogether to the democratisation of extravagance, but its impact truly is auxiliary to the impact of changing utilisation designs.The primary case we consider includes symmetric exchange costs on all joins, so that the two firms will each embrace the same number of plants, and either both or not one or the other will have a plant in S and, when they do, the trade designs of those plants will be symmetric. Third-country trade-stage generation emerges when moo generation costs in S lead a firm to utilise S to serve the other N nation, but the investment funds on settled costs from closing the domestic plant don't balanced the costs of shipping components to S and last yield back domestic.The US is an insider nation with regard to the NAFTA nations, Canada and Mexico, whereas it is an untouchable nation with regard to the coordinates European showcase. The US has distant-coming to special exchanging assentions with Israel, but so has the EU. The nations in Southeast Asia are ordinary low-cost nations that, not at all like Mexico and the nations in Southern Europe, don't have a place to a territorial free-trade region. We in this way need to utilise Southeast Asian nations as a group against which to assess comes about for NAFTA and the EU. Israel will be indicated independently as well.Effective teach secure property rights. The universal economy is getting to be a knowledge-based economy; as a result, mental property rights are getting to be ever more imperative. Their assurance is essential since those who deliver merchandise and administrations can be enough remunerated. The esteem of property rights can rapidly be crushed unless governments uphold rights in this range (Divider et al., 2010). The degree to which outside undertakings can increment ventures in a nation depends on the degree to which that nation has made teach and arrangements for ensuring mental property rights, and a government's adequacy in implementing these rights influences the choices of outside undertakings to enter a host nation, and how they work once there. Frail mental property rights dishearten remote financial specialists, especially in technology-intensive divisions, and energise ventures in ventures that center on dispersion, that's , trading to the host nation, instead of setting up generation plants that might give FDI spillovers; (Rondinelli, 2005).Promoting such offerings within the Joined together States moreover requires another move by firms and brands, that's , numerous analysts and indeed a few professionals respect extravagance shoppers as homogeneous (Dubois 1992). However in Europe, for illustration, unobtrusive social and behavioural assortment and contrasts rise among German, Danish, French, Spanish, and Irish extravagance buyers, likely due to the solid French and Italian impacts that regularly get forced on other European national markets. Extravagance utilisation behaviour can moreover changeessentially between regions, way of life, expendable wage, and over age bunches (Silverstein and Fiske 2003a).The new topic elegant and cool communicates personal taste, separates the respondent from others, and makes a difference illustrate modernity, observing capacities, or victory. Buyers in this way utilise items to send signals around who they are and their objectives. This topic moreover recognises the wants to seem “hip,” see in vogue, and feel special. Unmentionables, design extras, observes, and gems empower the participants, as buyers, to specific their style, information, taste, and values. In spite of the fact that our members don't show up driven basically by a crave for status, nor do they express uncritical captivation with a brand title, they still care almost the messages that merchandise and brands provide approximately their person fashion.The GDP development variable is positive and noteworthy. This suggests that outside speculators are pulled in to have nations with huge markets (market-seeking FDI). This finding is reliable with Bannaga et al. (2013).Trade openness variable is positive and critical. Potential remote speculators are able to gotten to be well educated approximately the nearby conditions of their worldwide exchange accomplices when exchange is freed. Subsequently, outside financial specialists favour contributing in nations with sizable exchange volume. This result is in line with Elfakhani and Matar (2007).Middle easterner economies ought to decrease political hazard by democratising their political frameworks, which may offer assistance settle the locale. They may accomplish legislative soundness by following the equitable change to worldwide community benchmarks in terms of regard for crucial human rights, reinforcing the role of law and advancing national decisions. Middle easterner economies ought to control debasement by supporting the run the show of law and building a successful, unbiased and straightforward legitimate framework. The successful procedure to upgrade the run the show of law requires the foundation of a legal framework in which all people, teach and indeed the government are responsible to laws that are consistently implemented and freely settled. Speculation legislations must secure and ensure ventures within the locale, and ought to be coherent with worldwide guidelines to extend certainty and decrease the risk of apportionment or nationalisation of remote speculations. There's a requirement for the locale to advance law based responsibility by receiving a administration framework based on regulation commitments in giving straightforward data almost open decisions and empowering gracious society to redress or authorise choices that don't follow to the order of law. Constitutions ought to avoid political parties from setting up or getting to be included in military strengths, to avoid the dangers of take-over of a chosen government or of debilitating its capacity to operate viably. LaDuke’s organisation restricts the proposed course for the modern pipeline, dreading that development and potential spills along the modern course would debilitate the wellbeing of watersheds and Ojibwe lands in Minnesota. “You don’t get to take off a mess in put for the state of Minnesota and tribes, who are exceptionally affected by this since it crosses three reservations, to clean up,” says LaDuke.In Canada, the modern Line 3 has as of now been built. The ancient line will be cleaned out and surrendered in put another year. The country’s National Vitality Board directs pipelines that cross borders and requires companies to yield an arrangement some time recently abandoning a pipeline that incorporates how they’ll set aside reserves to screen the decommissioned pipeline and pay attention of any issues that emerge. It’s a arrangement that’s missing within the US, in spite of the fact that the US Pipeline and Dangerous Materials Security Organisation requires deserted pipelines cleared out in put to be detached, cleansed of unsafe materials, and fixed.An evaluated 140 million individuals misplaced their employments after India bolted down its economy in Walk to halt the flare-up, whereas numerous others saw their pay rates definitely decreased, the Mumbai-based Center for Observing Indian Economy said. As the lockdown was facilitated, numerous went back to work, but more than six million individuals who misplaced occupations haven’t found unused employment.In a June study by the All India Producers Organisation, approximately one-third of little and medium-sized undertakings shown that their businesses were past sparing. The industry bunch said that such a “mass pulverisation of business” was unprecedented.As a run the show, they are created utilising physical strategies. For occurrence, much appreciated to the most modest high-frequency electron developments, the electrical resistance of a wire isn't consistent but instep vacillates marginally in an unpredictable way. Meaning estimations of this foundation commotion can be utilised to produce genuine irregular numbers.Presently, for the primary time, an investigation group driven by Robert Grass, Teacher at the Established of Chemical and Bioengineering, has portrayed a non-physical strategy of producing such numbers: one that employments biochemical signals and actually works in hone. Within the past, the thoughts put forward by other researchers for producing arbitrary numbers by chemical implies tended to be generally hypothetical.It’s all the more vital since the long run for oil and gas companies is more questionable than ever, which makes long-term of their pipelines more questionable, as well. The time will come when the US will got to figure with the destiny of all its pipelines — and when that time comes, the companies that worked them seem now not be around to bargain with them. The COVID-19 widespread alone sent oil and gas companies faltering toward insolvency. Indeed after the economy recoups from the pandemic, there will be more challenges ahead. Worldwide endeavours to handle climate alter, embraced within the point of interest 2015 Paris climate agreement, set countries on a way to cut down nursery gas outflows from burning fossil fills to net zero by 2050. Meaning a future with more renewables and small to no oil and gas.In long haul as unrefined oil request diminishes, we would anticipate to see increasingly pipelines being abandoned,” says Blackburn. “With the oil industry having less and less cash within the future, the hazard gets to be more noteworthy and more prominent that they’ll fair walk absent from their pipelines and take off landowners holding the bag.In any case, an examination appeared that the conveyance of the four building squares A, C, G and T was not totally indeed. Either the complexities of nature or the amalgamation strategy conveyed driven to the bases G and T being coordinates more habitually within the atoms than A and C. Inany case, the researchers were able to adjust this predisposition with a straightforward calculation, subsequently producing idealise irregular numbers.The most point of ETH Teacher Grass and his group was to appear that arbitrary events in chemical response can be misused to produce culminate irregular numbers. Interpreting the finding into a coordinate application was not a prime concern at to begin with. \"Compared with other strategies, be that as it may, our own has the advantage of being able to produce colossal amounts of haphazardness that can be put away in an amazingly little space, a single test tube,\" Grass says. \"Able to examined out the data and reinterpret it in computerised shape at a afterward date. Usually incomprehensible with the past methods.”India’s economy shrank 7.5 percent within the three months that finished in September compared with a year prior, government figures appeared on Friday. The information reflects the extending of India’s severest subsidence since at slightest 1996, when the nation to begin with started distributing its net household item numbers.The unused figures immovably hidden India’s position among the world’s worst-performing major economies, in spite of broad government investing outlined to protect the thousands of little businesses seriously battered by its long, quickly forced lockdown.Material manufacturing plants, calfskin tanneries, brick ovens, foundries and other little ventures shape “part of the country’s social texture, bringing nearby riches and neighborhood employment,” said Venkatachalam Anbumozhi, an financial specialist who centers on south and East Asia. Just many a long time back, India, with a populace of 1.3 billion individuals, was one of the world’s fastest-growing huge economies. It frequently clocked development of 8 percent or more.Global businesses started to warm to the thought of India as a potential substitute to China, both as a put to create merchandise and to offer them. China’s costs are rising, and its exchange war with the Joined together States has complicated doing trade there. The Chinese Communist Party is progressively interfering into commerce things, and nearby Chinese competitors have increased their amusement against universal brands."]

samp_candidate_long = ["The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands. The importance of export-platform FDI is documented in a study by Hanson, Mataloni, and Slaughter (2001). Using data on the foreign operations of US multinationals, they report that although the average share of exports in affiliate sales has remained constant at about one-third, there has been a substantial increase in Mexico and Canada after the formation of NAFTA. Their econometric analysis suggests that export platform FDI is promoted by low host-country trade barriers and discouraged by large host-country markets.In the past decade, the global marketplace has experienced the democratization of luxury, driven by a constantly growing aspirational consumer segment. This popular form of new luxury (Silverstein and Fiske 2003a) focuses on human involvement, self-expression, unrestricted distribution, and recognition from others. New luxury, also referred to as “masstige” (i.e., mass prestige; Cornell 2002), challenges conventional beliefs about luxury marketing and branding, including pricing and price points, brand extensions, consumer demand, sophistication, market stability, and the time needed to reach the middle class. The marketing of luxury has contributed significantly to the democratization of luxury, but its effect really is secondary to the influence of changing consumption patterns.The first case we consider involves symmetric trade costs on all links, so that the two firms will each adopt the same number of plants, and either both or neither will have a plant in S and, when they do, the export patterns of those plants will be symmetric. Third-country export-platform production arises when low production costs in S lead a firm to use S to serve the other N country, but the savings on fixed costs from closing the home plant do not offset the costs of shipping components to S and final output back home.The US is an insider country with respect to the NAFTA countries, Canada and Mexico, while it is an outsider country with respect to the integrated European market. The US has far-reaching preferential trading agreements with Israel, but so has the EU. The countries in Southeast Asia are typical low-cost countries that, unlike Mexico and the countries in Southern Europe, do not belong to a regional free-trade area. We thus want to use Southeast Asian countries as a group against which to evaluate results for NAFTA and the EU. Israel will be specified separately as well.Efficient institutions protect property rights. The international economy is becoming a knowledge- based economy; as a result, intellectual property rights are becoming ever more important. Their protection is necessary because those who produce goods and services can be adequately rewarded. The value of property rights can quickly be destroyed unless governments enforce rights in this area (Wall et al., 2010). The degree to which foreign enterprises can increase investments in a country relies on the degree to which that country has created institutions and policies for protecting intellectual property rights, and a government's effectiveness in enforcing these rights affects the decisions of foreign enterprises to enter a host country, and how they operate once there. Weak intellectual property rights discourage foreign investors, particularly in technology-intensive sectors, and encourage investments in projects that focus on distribution, that is, exporting to the host country, rather than establishing production plants that could provide FDI spillovers; (Rondinelli, 2005).Marketing such offerings in the United States also requires another shift by firms and brands, that is, many researchers and even some practitioners regard luxury consumers as homogeneous (Dubois 1992). Yet in Europe, for example, subtle cultural and behavioral variety and differences emerge among German, Danish, French, Spanish, and Irish luxury consumers, likely due to the strong French and Italian influences that often get imposed on other European national markets. Luxury consumption behavior can also vary significantly between localities, lifestyle, disposable income, and across age groups (Silverstein and Fiske 2003a).The emergent theme fashionable and cool expresses personal taste, differentiates the respondent from others, and helps demonstrate sophistication, discerning abilities, or success. Consumers thus use products to send signals about who they are and their goals. This theme also acknowledges the desires to appear “hip,” look stylish, and feel unique. Lingerie, fashion accessories, watches, and jewelry enable the participants, as consumers, to express their style, knowledge, taste, and values. Although our participants do not appear driven primarily by a desire for status, nor do they express uncritical infatuation with a brand name, they still care about the messages that goods and brands deliver about their individual style.The GDP growth variable is positive and significant. This implies that foreign investors are attracted to host countries with large markets (market-seeking FDI). This finding is consistent with Bannaga et al. (2013).Trade openness variable is positive and significant. Potential foreign investors are able to become well informed about the local conditions of their international trade partners when trade is liberated. Therefore, foreign investors prefer investing in countries with sizeable trade volume. This result is in line with Elfakhani and Matar (2007).Arab economies need to reduce political risk by democratising their political systems, which may help stabilise the region. They could achieve governmental stability by adhering the democratic transformation to international community benchmarks in terms of respect for fundamental human rights, strengthening the role of law and promoting national elections. Arab economies need to control corruption by supporting the rule of law and building an effective, impartial and transparent legal system. The effective strategy to enhance the rule of law requires the establishment of a judicial system in which all individuals, institutions and even the government are accountable to laws that are uniformly enforced and independently adjudicated. Investment legislations must protect and guarantee investments in the region, and should be coherent with global standards to increase confidence and reduce the risk of appropriation or nationalisation of foreign investments. There is a need for the region to promote democratic accountability by adopting a governance system based on institutional obligations in providing transparent information about public decisions and enabling civil society to correct or sanction decisions that do not adhere to the mandate of law. Constitutions should prevent political parties from establishing or becoming involved in military forces, to avoid the risks of take-over of an elected government or of weakening its ability to function effectively. LaDuke’s organization opposes the proposed route for the new pipeline, fearing that construction and potential spills along the new route would threaten the health of watersheds and Ojibwe lands in Minnesota. “You don’t get to leave a mess in place for the state of Minnesota and tribes, who are very impacted by this because it crosses three reservations, to clean up,” says LaDuke.In Canada, the new Line 3 has already been built. The old line will be cleaned out and abandoned in place next year. The country’s National Energy Board oversees pipelines that cross borders and requires companies to submit a plan before abandoning a pipeline that includes how they’ll set aside funds to monitor the decommissioned pipeline and take care of any problems that arise. It’s a provision that’s lacking in the US, although the US Pipeline and Hazardous Materials Safety Administration requires abandoned pipelines left in place to be disconnected, purged of hazardous materials, and sealed.An estimated 140 million people lost their jobs after India locked down its economy in March to stop the outbreak, while many others saw their salaries drastically reduced, the Mumbai-based Center for Monitoring Indian Economy said. As the lockdown was eased, many went back to work, but more than six million people who lost jobs haven’t found new employment.In a June survey by the All India Manufacturers Organization, about one-third of small and medium-sized enterprises indicated that their businesses were beyond saving. The industry group said that such a “mass destruction of business” was unprecedented.As a rule, they are generated using physical methods. For instance, thanks to the tiniest high- frequency electron movements, the electrical resistance of a wire is not constant but instead fluctuates slightly in an unpredictable way. That means measurements of this background noise can be used to generate true random numbers.Now, for the first time, a research team led by Robert Grass, Professor at the Institute of Chemical and Bioengineering, has described a non-physical method of generating such numbers: one that uses biochemical signals and actually works in practice. In the past, the ideas put forward by other scientists for generating random numbers by chemical means tended to be largely theoretical.It’s all the more important because the future for oil and gas companies is more uncertain than ever, which makes the future of their pipelines more uncertain, too. The time will come when the US will need to reckon with the fate of all its pipelines — and when that time comes, the companies that operated them could no longer be around to deal with them. The COVID-19 pandemic alone sent oil and gas companies stumbling toward bankruptcy. Even after the economy recovers from the pandemic, there will be more challenges ahead. Global efforts to tackle climate change, adopted in the landmark 2015 Paris climate accord, set nations on a path to cut down greenhouse gas emissions from burning fossil fuels to net zero by 2050. That means a future with more renewables and little to no oil and gas.“In the future as crude oil demand decreases, we would expect to see more and more pipelines being abandoned,” says Blackburn. “With the oil industry having less and less money in the future, the risk becomes greater and greater that they’ll just walk away from their pipelines and leave landowners holding the bag.”However, an analysis showed that the distribution of the four building blocks A, C, G and T was not completely even. Either the intricacies of nature or the synthesis method deployed led to the bases G and T being integrated more frequently in the molecules than A and C. Nonetheless, the scientists were able to correct this bias with a simple algorithm, thereby generating perfect random numbers.The main aim of ETH Professor Grass and his team was to show that random occurrences in chemical reaction can be exploited to generate perfect random numbers. Translating the finding into a direct application was not a prime concern at first. \"Compared with other methods, however,ours has the advantage of being able to generate huge quantities of randomness that can be stored in an extremely small space, a single test tube,\" Grass says. \"We can read out the information and reinterpret it in digital form at a later date. This is impossible with the previous methods.”India’s economy shrank 7.5 percent in the three months that ended in September compared with a year earlier, government figures showed on Friday. The data reflects the deepening of India’s severest recession since at least 1996, when the country first began publishing its gross domestic product numbers.The new figures firmly ensconced India’s position among the world’s worst- performing major economies, despite expansive government spending designed to rescue the thousands of small businesses severely battered by its long, hastily imposed lockdown.Textile factories, leather tanneries, brick kilns, foundries and other small enterprises form “part of the country’s social fabric, bringing local wealth and local employment,” said Venkatachalam Anbumozhi, an economist who focuses on South and East Asia.Just a few years ago, India, with a population of 1.3 billion people, was one of the world’s fastest-growing large economies. It regularly clocked growth of 8 percent or more.Global businesses began to warm to the idea of India as a potential substitute to China, both as a place to make goods and to sell them. China’s costs are rising, and its trade war with the United States has complicated doing business there. The Chinese Communist Party is increasingly intruding into business matters, and local Chinese competitors have upped their game against international brands."]

#File Parsing

#Preprocessing 1

#Engines & Content Scraping

#Preprocessing 2

#Level 1: Common Substrings
Will potentially be excluded

In [ ]:
samp = ['the first season of the military science fiction television series stargate sg-1 commenced airing on the showtime channel in the united states on july 27, 1997, concluded on the sci fi channel on march 6, 1998, and contained 22 episodes.',
  'the first season of the scifi science fiction television series began airing on the showtime channel in the united states in 1997 and ended on thesci fi channel in 1998 the first season.']

STRIDE = 3

In [ ]:
def find_common_substrings(text_suspect:str, text_candidate:str, ngram_size:int=3, stride:int=1, skip_stride=True, exclude_single_ngrams=True):
  '''
  Extracts all common substrings of size S from the input strings
  {text_suspect, text_candidate} idle
  {ngram_size, stride} idle
  {skip_stride} combine closely-consecutive or only true-consecutive ngrams into a single passage (CURRENTLY WORKS ONLY BECAUSE NGRAM_SIZE=3 AND STRIDE=1)
  {exclude_single_ngrams} to exclude single orphans if ngrams size is small enough (SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING)
  -----
  %output: array of {passage_suspect:str, passage_candidate:str,
                     range_suspect: (int, int), range_candidate: (int, int)}
  '''

  shingles_suspect = shingles(text_suspect, ngram_size=ngram_size, stride=stride)
  shingles_candidate = shingles(text_candidate, ngram_size=ngram_size, stride=stride)

  intersection = set(shingles_suspect.values()) & set(shingles_candidate.values())
  #Convert common shingles into shingle positions in each of suspect and candidate
  intersection_positions = defaultdict(list) #{position_in_suspect: [positions_in_candidate]}
  intersection_positions_list = []

  for shingle in intersection:
    keys_suspect = [k for k, v in shingles_suspect.items() if shingle == v]
    keys_candidate = [k for k, v in shingles_candidate.items() if shingle == v]
    for key_suspect, key_candidate in product(keys_suspect, keys_candidate):
      intersection_positions_list.append((key_suspect, key_candidate))
      intersection_positions[key_suspect].append(key_candidate)

  #Combine consecutive keys to form ranges
  intersection_positions_list = sorted(intersection_positions_list, key=lambda x: x[0])
  intersection_positions_list

  _cpt = 0
  series = defaultdict(list)
  cache = dict()
  for i, j in intersection_positions_list:
    if (i-1, j-1) in intersection_positions_list:
      index = cache[(i-1, j-1)]
      series[index].append((i, j))
      cache[(i, j)] = index
      #!!!!!!!!!!!!!!!!!!!!!!! Beware, the following works only because ngram_size == 3 and stride == 1 !!!!!!!!!!!!!!!!!!!!!!!#
    elif skip_stride and (i-(ngram_size+1), j-(ngram_size+1)) in intersection_positions_list: #if word in between was modified, combine suites if single word is missing (i.e if distance == ngram_size)
      index = cache[(i-(ngram_size+1), j-(ngram_size+1))]
      series[index].append((i, j))
      cache[(i, j)] = index
    else:
      series[_cpt].append((i, j))
      cache[(i, j)] = _cpt
      _cpt += 1

  #Remove orphan, single pairs if ngram_size is small enough
  if ngram_size <= SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING and exclude_single_ngrams:
    for key in list(series.keys()): #cast to list to force a copy and avoid RuntimeError: dictionary changed size during iteration
      if len(series[key]) < 2:
        series.pop(key)

  #transform series back into
  result = []
  for serie in series.values():
    pos_sus = (serie[0][0], serie[-1][0])
    pos_can = (serie[0][1], serie[-1][1])

    passage_sus = shingles_1[serie[0][0]]
    passage_can = shingles_2[serie[0][1]]

    #adding only the last word of the shingle, regardless of stride size
    passage_sus += " " + " ".join([shingles_1[x].split()[-1] for x in [y for y in range(pos_sus[0]+1,pos_sus[1]+1)]])

    passage_can += " " + " ".join([shingles_2[x].split()[-1] for x in [y for y in range(pos_can[0]+1,pos_can[1]+1)]])

    item = {
        'passage_suspect': passage_sus,
        'passage_candidate': passage_can,
        'range_suspect': (serie[0][0], serie[-1][0]),
        'range_candidate': (serie[0][1], serie[-1][1]),
    }
    result.append(item)

  return result


In [ ]:
text_1 = samp[0]
text_2 = samp[1]
result = find_common_substrings(text_1, text_2, skip_stride=True)

In [ ]:
result

[{'passage_candidate': 'the first season of the scifi science fiction television series',
  'passage_suspect': 'the first season of the military science fiction television series',
  'pos_candidate': (0, 7),
  'pos_suspect': (0, 7)},
 {'passage_candidate': 'airing on the showtime channel in the united states',
  'passage_suspect': 'airing on the showtime channel in the united states',
  'pos_candidate': (11, 17),
  'pos_suspect': (14, 20)}]

In [ ]:
result

[{'passage_candidate': 'the first season of the',
  'passage_suspect': 'the first season of the',
  'pos_candidate': (0, 2),
  'pos_suspect': (0, 2)},
 {'passage_candidate': 'science fiction television series',
  'passage_suspect': 'science fiction television series',
  'pos_candidate': (6, 7),
  'pos_suspect': (6, 7)},
 {'passage_candidate': 'airing on the showtime channel in the united states',
  'passage_suspect': 'airing on the showtime channel in the united states',
  'pos_candidate': (11, 17),
  'pos_suspect': (14, 20)}]

In [ ]:
 for i in positions:
    if (i-1) in positions:
      #print("Flush")
      index = cache[i-1]
      suites[index].append(i)
      cache[i] = index
    elif skip_stride and any(x in positions for x in range(i-int(ngram_size/ngram_stride), i-1)): #if word in between was modified, combine suites if single word is missing (i.e if distance == STRIDE)
      #print("Close Flush")
      for x in range(i-ngram_stride, i-1): #to find which one satisfied the condition first (smaller)
        if x in positions:
          index = cache[x]
          suites[index].append(i)
          cache[i] = index
          break
    else:
      suites[cpt].append(i)
      cache[i] = cpt
      cpt += 1

[{'passage_candidate': 'the first season of the',
  'passage_suspect': 'the first season of the',
  'pos_candidate': (0, 2),
  'pos_suspect': (0, 2)},
 {'passage_candidate': 'science fiction television series',
  'passage_suspect': 'science fiction television series',
  'pos_candidate': (6, 7),
  'pos_suspect': (6, 7)},
 {'passage_candidate': 'airing on the showtime channel in the united states',
  'passage_suspect': 'airing on the showtime channel in the united states',
  'pos_candidate': (11, 17),
  'pos_suspect': (14, 20)}]

#Level 2: Jaccard + Fastext

###Level 2 common utility functions

In [49]:
import re
from nltk.util import ngrams
from itertools import product
from collections import OrderedDict, defaultdict
from typing import List
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
!pip install fasttext
import fasttext

     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093648 sha256=4a57c6d26e41c6c2a1b957e06229256211999bf11bc47dc5aa90652e9146233e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


####Acquiring Fasttext pre-trained weights

In [48]:
#English model
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
#!gunzip /content/drive/MyDrive/Plagiarism\ Detection/cc.en.300.bin.gz
!gunzip /content/cc.en.300.bin.gz
fasttext_model_en = '/content/drive/MyDrive/Plagiarism Detection/cc.en.300.bin'

#French model
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
#!gunzip /content/drive/MyDrive/Plagiarism Detection/cc.fr.300.bin.gz
fasttext_model_fr = '/content/drive/MyDrive/Plagiarism Detection/cc.fr.300.bin'

#Arabic model
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.bin.gz
#!gunzip /content/drive/MyDrive/Plagiarism\ Detection/cc.ar.300.bin.gz
fasttext_model_ar = '/content/drive/MyDrive/Plagiarism Detection/cc.ar.300.bin'

--2021-07-03 19:40:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  27.3MB/s    in 2m 37s  

2021-07-03 19:43:00 (27.3 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



####Functions

In [10]:
def shingles(text, ngram_size, stride):
  '''
  Generate shingles (overlapping n-grams) from input
  {text} standard string of text (not preprocessed)
  {size} n, size of n-grams
  {stride} number of skipped tokens between consecutive shingles, (stride=1) is true overlapping n-grams
  -----
  %output: dict of {position:int, shingle:str}
  '''
  text = text.lower()
  text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
  tokens = [token for token in text.split(" ") if token != ""]
  shingles = list(" ".join(i) for i in ngrams(tokens, ngram_size))
  strided_shingles = shingles[0::stride]
  #return dict(zip(range(1, len(shingles)), shingles))
  return dict(zip(range(0, len(strided_shingles)), strided_shingles))

In [28]:
def contains_overlaping_sub_ranges(ranges: List[Tuple[Tuple[int, int], Tuple[int, int]]]):
  '''
  Checks the state of a list of positional ranges. State is related 
  to the presence of overlap between at least one pair of ranges.

  *Ranges of sorted by default*

  :param ranges: list of (suspect_range, candidate_range) positional ranges. 
  :returns: input contains at least one pair of consecutive overlaping ranges
  '''
  for i in range(len(ranges) - 1):
    range_sus, range_can = ranges[i]
    range_sus_next, range_can_next = ranges[i+1]
    if (range_sus[1] >= range_sus_next[0] - SKIP_TOLERANCE) and (range_can[1] >= range_can_next[0] - SKIP_TOLERANCE):# and (((range_can[1] - range_can_next[0]) - (range_sus[1] - range_sus_next[0])) <= SKIP_TOLERANCE):
      return True
  return False

#merge_ranges_overlap
def merge_sub_ranges(ranges: List[Tuple[Tuple[int, int], Tuple[int, int]]]):
  '''
  Recursive, Merges all pairs of overlaping consecutive positional ranges and checks for new state.

  *Ranges are sorted by default*
  :param ranges: list of (suspect_range, candidate_range) positional ranges. 
  :returns: list of positional ranges free of overlap
  '''
  if len(ranges) > 1:
    non_overlaping_ranges = []
    i = 0
    while i in range(len(ranges) - 1):
      range_sus, range_can = ranges[i]
      range_sus_next, range_can_next = ranges[i+1]
      if (range_sus[1] >= range_sus_next[0] - SKIP_TOLERANCE) and (range_can[1] >= range_can_next[0] - SKIP_TOLERANCE):# and (((range_can[1] - range_can_next[0]) - (range_sus[1] - range_sus_next[0])) <= SKIP_TOLERANCE):
        new_range_sus = (min(range_sus[0], range_sus_next[0]), max(range_sus[1], range_sus_next[1]))
        new_range_can = (min(range_can[0], range_can_next[0]), max(range_can[1], range_can_next[1]))
        
        non_overlaping_ranges.append((new_range_sus, new_range_can))
        i += 1 #skip next element because it was merged
      else:
        non_overlaping_ranges.append((range_sus, range_can))
        if i == len(ranges) - 2: #to deal with last element
          non_overlaping_ranges.append((range_sus_next, range_can_next))
      i += 1
    if contains_overlaping_sub_ranges(non_overlaping_ranges):
      non_overlaping_ranges = merge_sub_ranges(non_overlaping_ranges)
    return non_overlaping_ranges
  else:
    return ranges

In [59]:
def contains_overlaping_ranges(ranges):
  '''
  Checks the state of a list of positional ranges. State is related 
  to the presence of overlap between at least one pair of ranges.

  *Ranges of sorted by default*

  :param ranges: list of (suspect_range, candidate_range) positional ranges. 
  :returns: input contains at least one pair of consecutive overlaping ranges
  '''
  result = []
  for _range in ranges:
    possibilities = [item for item in ranges[ranges.index(_range):] if ((item[0][0] - SKIP_TOLERANCE <= _range[0][1]) and (item[1][0] - SKIP_TOLERANCE <= _range[1][1]) and (item[1][1] >= _range[1][0]))]
    if len(possibilities) > 1:
      return True
  return False


def merge_ranges(ranges):
  '''
  Recursive, Merges all pairs of overlaping consecutive positional ranges and checks for new state.

  *Ranges are sorted by default*

  :param ranges: list of (suspect_range, candidate_range) positional ranges. 
  :returns: list of positional ranges free of overlap
  '''
  if len(ranges) > 1:
    copy_ranges = ranges.copy()
    result = []
    while len(copy_ranges) > 0:
      first_item = copy_ranges[0]
      possibilities = [item for item in copy_ranges if ((item[0][0] - SKIP_TOLERANCE <= first_item[0][1]) and (item[1][0] - SKIP_TOLERANCE <= first_item[1][1]) and (item[1][1] >= first_item[1][0]))]
      #ic(first_item, possibilities)
      merged = merge_sub_ranges(possibilities)
      for item in possibilities:
        copy_ranges.remove(item)
      for item in merged:
        result.append(item)

    if contains_overlaping_ranges(result):
        result = merge_ranges(result)
    return result
  else:
    return ranges

#((2, 2), (1717, 1717)), ((3, 13), (2, 12))
#((2, 13), (2, 1717))

In [111]:
def merge_shingles(suspect_candidate_pairs_positions, suspect_shingles, candidate_shingles, plag_type:PlagType, skip_stride=True, ngram_stride:int=7, ngram_size:int=15, exclude_single_ngrams=False):
  '''
  Parse and merge consecutive highlighted shingles based on their position in the source text
    {suspect_candidate_pairs_positions} list of pair positions (sus, can) of the highlighted shingles from suspect and candidate
    {suspect_shingles, candidate_shingles} complete list of shingles of the source texts (not just the highlighted ones)
    {skip_stride} since shingles overlap, this options will force to combine non consecutive shingles that are close enough, eith distance <= ngram_size/ngram_stride
                  If false, only true consecutive shingles will be combined (may increase granularity)
    {ngram_size} as defined for [shingles] function
    {ngram_stride} as defined for [shingles] function
    {exclude_single_ngrams} to exclude single orphans if ngrams size is small enough (SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING)
  -----
  %output: 
    1- array of {suspect_range:(start, end), suspect_passage:str, candidate_range:(start, end), candidate_passage:str}
    2- total size of the passages (for percentage calculation, will be removed)
  '''
  #build chains of consecutive, or forgivingly consecutive shingles
  ic("EXCLUDE SINGLE SUSPECT SHINGLES THAT APPEAR IN ANOTHER RANGE")
  _cpt = 0
  series = defaultdict(list)
  cache = dict()
  for i, j in suspect_candidate_pairs_positions:
    if (i-1, j-1) in suspect_candidate_pairs_positions:
      index = cache[(i-1, j-1)]
      series[index].append((i, j))
      cache[(i, j)] = index
    elif skip_stride and any((x, y) in suspect_candidate_pairs_positions for (x, y) in product(range(i-int(ngram_size/ngram_stride), i-1), range(j-int(ngram_size/ngram_stride), j-1))): #if word in between was modified, combine suites if single word is missing (i.e if distance == ngram_size)
      for (x, y) in product(range(i-int(ngram_size/ngram_stride), i-1), range(j-int(ngram_size/ngram_stride), j-1)):
        #to find which one satisfied the condition first (smaller) 
        if (x, y) in suspect_candidate_pairs_positions:
          index = cache[(x, y)]
          series[index].append((i, j))
          cache[(i, j)] = index
          break
    else:
      series[_cpt].append((i, j))
      cache[(i, j)] = _cpt
      _cpt += 1

  #Remove orphan, single pairs if ngram_size is small enough
  if exclude_single_ngrams and ngram_size <= SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING:
    for key in list(series.keys()): #cast to list to force a copy and avoid RuntimeError: dictionary changed size during iteration
      if len(series[key]) < 2:
        series.pop(key)
  granularity = len(series)

  #Check for overlaping ranges
  pos_ranges = []
  for serie in series.values():
    pos_sus = (serie[0][0], serie[-1][0])
    pos_can = (serie[0][1], serie[-1][1])
    pos_ranges.append((pos_sus, pos_can))

  if contains_overlaping_ranges(pos_ranges):
    ic("found overlap")
    pos_ranges = merge_ranges(pos_ranges)
  granularity = len(pos_ranges)

  #Remove single suspect shingles that already belong in a range
  #for pos_range in pos_ranges:
    #if size = 1 and pos is included in previous or past range, delete element, or is equal to the range (if multiple single-ranges exist)


  #Remove duplicate ranges


  #transform ranges back into text
  instances = []
  instances_length = 0
  for pos_range in pos_ranges:
    pos_sus = pos_range[0]
    pos_can = pos_range[1]

    passage_sus = suspect_shingles[pos_sus[0]]
    passage_can = candidate_shingles[pos_can[0]]

    #adding only the last 'ngram_stride' words of the shingle
    passage_sus += " " + " ".join([" ".join(suspect_shingles[x].split()[-ngram_stride:]) for x in [y for y in range(pos_sus[0]+1,pos_sus[1]+1)]])

    passage_can += " " + " ".join([" ".join(candidate_shingles[x].split()[-ngram_stride:]) for x in [y for y in range(pos_can[0]+1,pos_can[1]+1)]])

    item = PlagInstance(pos_sus, pos_can, passage_sus, passage_can, plag_type)
    '''item = {
        'passage_suspect': passage_sus,
        'passage_candidate': passage_can,
        'range_suspect': pos_sus,
        'range_candidate': pos_can,
    }'''
    instances.append(item)
  return instances, granularity


In [31]:
def print_pairwise_matrix(matrix):
  '''
  Plots a matrix for visualisation
  '''
  fig, ax = plt.subplots(1,1, figsize=(20, 20))

  mask = np.triu(matrix)
  ax = sns.heatmap(matrix, ax=ax, vmin=0.0, vmax=1.0, cmap='afmhot', linewidths=.3)#, mask=mask)# annot=True, , cmap=cmap, linewidths=.5)
  ax.set_title("Jaccard Distance", color="black").set_fontsize(20)
  ax.tick_params(length=0)
  plt.xlabel("Suspect Document")
  plt.ylabel("Candidate Document")
  #plt.title("Fasttext - Sentence vectors")

  return plt

##Jaccard

In [32]:
def jaccard_words_overlap_distance(string_1, string_2, option='union'):
  '''
  Calculates jaccard distance (not similarity) between two input strings
    {string_1, string_2} idle
    {option} Jaccard denominator option, among [min, max, union]
  -----
  %output: float [.0, 1.0], jaccard distance
  '''

  if option not in ['min', 'max', 'union']:
    raise Exception("Jaccard Distance: option can only be [min, max, union], default to union.")
  
  str1 = set(string_1.split())
  str2 = set(string_2.split())

  if option == 'min':
    denominator = min(len(str1), len(str2))
  elif option == 'max':
    denominator = max(len(str1), len(str2))
  else: #default to union
    denominator = len(str1 | str2)

  try:
    unit_score = 1 - float(len(str1 & str2) / denominator)
  except: #sometimes, one sentence gets nullified by preprocessing
    unit_score = 0
  return unit_score

#TODO: Optimize matrix generation with numpy or others
def jaccard_pairwise_distance(suspect_shingles, candidate_shingles, option='union'):
  '''
  Generate pairwise jaccard distance matrix (not similarity) of size len(suspects) x len(candidates)
    {suspect_shingles, candidate_shingles} idle
    {option} Jaccard denominator option, among [min, max, union]
  -----
  %output: array of arrays, pairwise distance matrix
  '''
  matrix = np.zeros((len(suspect_shingles),len(candidate_shingles)),dtype=np.float)
  
  for i in range(len(suspect_shingles)):
    for j in range(len(candidate_shingles)):
        matrix[i,j] = jaccard_words_overlap_distance(suspect_shingles[i],candidate_shingles[j], option=option)
  return matrix

####Legacy

In [25]:
def fuse_consecutive_shingles(suspect_candidate_pairs_positions, suspect_shingles, candidate_shingles, skip_stride=True, ngram_stride:int=7, ngram_size:int=15, exclude_single_ngrams=False):
  '''
  Parse and combine consecutive highlighted shingles based on their position in the source text
    {suspect_candidate_pairs_positions} list of pair positions (sus, can) of the highlighted shingles from suspect and candidate
    {suspect_shingles, candidate_shingles} complete list of shingles of the source texts (not just the highlighted ones)
    {skip_stride} since shingles overlap, this options will force to combine non consecutive shingles that are close enough, eith distance <= ngram_size/ngram_stride
                  If false, only true consecutive shingles will be combined (may increase granularity)
    {ngram_size} as defined for [shingles] function
    {ngram_stride} as defined for [shingles] function
    {exclude_single_ngrams} to exclude single orphans if ngrams size is small enough (SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING)
  -----
  %output: 
    1- array of {suspect_range:(start, end), suspect_passage:str, candidate_range:(start, end), candidate_passage:str}
    2- total size of the passages (for percentage calculation, will be removed)
  '''
  #build chains of consecutive, or forgivingly consecutive shingles
  _cpt = 0
  series = defaultdict(list)
  cache = dict()
  for i, j in suspect_candidate_pairs_positions:
    if (i-1, j-1) in suspect_candidate_pairs_positions:
      index = cache[(i-1, j-1)]
      series[index].append((i, j))
      cache[(i, j)] = index
    elif skip_stride and any((x, y) in suspect_candidate_pairs_positions for (x, y) in product(range(i-int(ngram_size/ngram_stride), i-1), range(j-int(ngram_size/ngram_stride), j-1))): #if word in between was modified, combine suites if single word is missing (i.e if distance == ngram_size)
      for (x, y) in product(range(i-int(ngram_size/ngram_stride), i-1), range(j-int(ngram_size/ngram_stride), j-1)):
        #to find which one satisfied the condition first (smaller) 
        if (x, y) in suspect_candidate_pairs_positions:
          index = cache[(x, y)]
          series[index].append((i, j))
          cache[(i, j)] = index
          break
    else:
      series[_cpt].append((i, j))
      cache[(i, j)] = _cpt
      _cpt += 1

  #Remove orphan, single pairs if ngram_size is small enough
  if exclude_single_ngrams and ngram_size <= SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING:
    for key in list(series.keys()): #cast to list to force a copy and avoid RuntimeError: dictionary changed size during iteration
      if len(series[key]) < 2:
        series.pop(key)
  granularity = len(series)

  #transform series back into text
  result = []
  total_size = 0
  for serie in series.values():
    pos_sus = (serie[0][0], serie[-1][0])
    pos_can = (serie[0][1], serie[-1][1])

    passage_sus = suspect_shingles[serie[0][0]]
    passage_can = candidate_shingles[serie[0][1]]

    #adding only the last word of the shingle, regardless of stride size
    passage_sus += " " + " ".join([suspect_shingles[x].split()[-1] for x in [y for y in range(pos_sus[0]+1,pos_sus[1]+1)]])

    passage_can += " " + " ".join([candidate_shingles[x].split()[-1] for x in [y for y in range(pos_can[0]+1,pos_can[1]+1)]])

    item = {
        'passage_suspect': passage_sus,
        'passage_candidate': passage_can,
        'range_suspect': (serie[0][0], serie[-1][0]),
        'range_candidate': (serie[0][1], serie[-1][1]),
    }
    result.append(item)
    total_size += len(re.findall(r'\w+', passage_sus))

  return result, total_size, granularity


In [ ]:
def aggregate_report(suspect_candidate_pairs_positions, suspect_shingles, candidate_shingles, source_text_length:int, skip_stride=True, ngram_stride=7, ngram_size=15):
    '''
    Calculate total detected percentage after reducing granularity
      {suspect_candidate_pairs_positions} list of pair positions (sus, can) of the highlighted shingles from suspect and candidate
      {suspect_shingles, candidate_shingles} complete list of shingles of the source texts (not just the highlighted ones)
      {skip_stride} since shingles overlap, this options will force to combine non consecutive shingles that are close enough, eith distance <= ngram_size/ngram_stride
                If false, only true consecutive shingles will be combined (may increase granularity)
      {ngram_size} as defined for [shingles] function
      {ngram_stride} as defined for [shingles] function
      {exclude_single_ngrams} to exclude single orphans if ngrams size is small enough (SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING)
      {source_text_length} length of the original text (not shingled)

    -----
    %output: 
      1- array of {position_range:(start, end), passage:str}
      2- total size of the passages (for percentage calculation, will be removed)
    '''
    #unique_positions = list(dict.fromkeys(highlighted_shingles_positions))
    
    passages, passages_length, granularity = fuse_consecutive_shingles(suspect_candidate_pairs_positions, suspect_shingles, candidate_shingles, skip_stride=skip_stride, ngram_stride=ngram_stride, ngram_size=ngram_size)
    
    percentage = (float(passages_length)/source_text_length) * 100
    percentage = round(percentage, 2)
    
    print("Detected percentage = {}% (text = {} & detected = {})".format(percentage, source_text_length, passages_length))

    return passages, percentage, granularity

#-------------------------
 


In [ ]:
def jaccard_words_overlap_distance(string_1, string_2, option='union'):
  '''
  Calculates jaccard distance (not similarity) between two input strings
    {string_1, string_2} idle
    {option} Jaccard denominator option, among [min, max, union]
  -----
  %output: float [.0, 1.0], jaccard distance
  '''

  if option not in ['min', 'max', 'union']:
    raise Exception("Jaccard Distance: option can only be [min, max, union], default to union.")
  
  str1 = set(string_1.split())
  str2 = set(string_2.split())

  if option == 'min':
    denominator = min(len(str1), len(str2))
  elif option == 'max':
    denominator = max(len(str1), len(str2))
  else: #default to union
    denominator = len(str1 | str2)

  try:
    unit_score = 1 - float(len(str1 & str2) / denominator)
  except: #sometimes, one sentence gets nullified by preprocessing
    unit_score = 0
  return unit_score

#TODO: Optimize matrix generation with numpy or others
def jaccard_pairwise_distance(suspect_shingles, candidate_shingles, option='union'):
  '''
  Generate pairwise jaccard distance matrix (not similarity) of size len(suspects) x len(candidates)
    {suspect_shingles, candidate_shingles} idle
    {option} Jaccard denominator option, among [min, max, union]
  -----
  %output: array of arrays, pairwise distance matrix
  '''
  matrix = np.zeros((len(suspect_shingles),len(candidate_shingles)),dtype=np.float)

  for i in range(len(suspect_shingles)):
    for j in range(len(candidate_shingles)):
        matrix[i,j] = jaccard_words_overlap_distance(suspect_shingles[i],candidate_shingles[j], option=option)

  return matrix

In [ ]:
def jaccard_subreport(suspect_text, candidate_text, threshold:float=0.25, ngram_size:int=15, ngram_stride:int=7, option='union'):
  '''
  Generate global similarity detection report
    {suspect_text, candidate_text} standard string of text (not preprocessed)
    {threshold} value under which the jaccard distance indicates a pair of strings to be similar. Smaller is more similar
    {ngram_size} as defined for [shingles] function
    {ngram_stride} as defined for [shingles] function
    {option} Jaccard denominator option, among [min, max, union]
  -----
  %output: 
    1- array of {
        suspect_pos:int, suspect_text:str, 
        candidate_pos:int, candidate_text:str
        jaccard_distance:float
        }
    2- dict of input hyper-parameters just for easy association (may be deleted later)
  '''
  print("ADD SECOND THRESHOLD FOR WHAT TO PASS TO FASTEXT?")
  print("INPUT SHINGLES INSTEAD OF TEXT TO BE SHINLGED (because same input as for Fasttext)")

  if (not isinstance(ngram_size, int)) or (not isinstance(ngram_stride, int)) or (threshold >= 1):
    raise Exception("Error, incorrect parameters for jaccard_subreport:\n- N-Gram size: {} (should be int [1;~])\n- Stride: {} (should be int [1;~])\n- Threshold: {} (should be float [.0;1.0[)".format(ngram_size, ngram_stride, threshold))

  suspect_shingles = shingles(suspect_text, ngram_size=ngram_size, stride=ngram_stride)
  candidate_shingles = shingles(candidate_text, ngram_size=ngram_size, stride=ngram_stride)
  
  matrix = jaccard_pairwise_distance(suspect_shingles, candidate_shingles, option=option)
  mask = np.where(matrix < threshold)

  PAIRS = []
  for x, y in zip(mask[0], mask[1]):
    pair = {
        'suspect_pos': x,
        'suspect_text': suspect_shingles[x],
        'candidate_pos': y,
        'candidate_text': candidate_shingles[y],
        'jaccard_distance': matrix[x][y]
    }
    PAIRS.append(pair)

  #unique_suspect_positions = [pair['suspect_pos'] for pair in PAIRS]
  suspect_text_length = len(re.findall(r'\w+', suspect_text))
  
  #percentage, detected_passages, number_passages = aggregate_report(unique_suspect_positions, suspect_shingles, suspect_text_length)
  _pairs = [(pair['suspect_pos'], pair['candidate_pos']) for pair in PAIRS]
  PASSAGE_PAIRS, percentage, granularity = aggregate_report(_pairs, suspect_shingles, candidate_shingles, suspect_text_length)

  PARAMETERS = {'n-gram_size': ngram_size, 'n-gram_stride': ngram_stride, 'threshold': threshold, 'percentage': percentage, 'granularity': granularity}

  return PAIRS, PARAMETERS, PASSAGE_PAIRS

####Working

In [33]:
def jaccard_subreport(suspect_shingles, candidate_shingles, suspect_text_length, threshold_semant:float=0.5, threshold:float=0.25, ngram_size:int=15, ngram_stride:int=7, option='union', print_matrix=False):
  '''
  Generate global similarity detection report
    {suspect_text, candidate_text} standard string of text (not preprocessed)
    {threshold} value under which the jaccard distance indicates a pair of strings to be similar. Smaller is more similar
    {ngram_size} as defined for [shingles] function
    {ngram_stride} as defined for [shingles] function
    {option} Jaccard denominator option, among [min, max, union]
  -----
  %output: 
    1- array of {
        suspect_pos:int, suspect_text:str, 
        candidate_pos:int, candidate_text:str
        jaccard_distance:float
        }
    2- dict of input hyper-parameters just for easy association (may be deleted later)
  '''
  if (not isinstance(ngram_size, int)) or (not isinstance(ngram_stride, int)) or (threshold >= 1):
    raise Exception("Error, incorrect parameters for jaccard_subreport:\n- N-Gram size: {} (should be int [1;~])\n- Stride: {} (should be int [1;~])\n- Threshold: {} (should be float [.0;1.0[)".format(ngram_size, ngram_stride, threshold))

  matrix = jaccard_pairwise_distance(suspect_shingles, candidate_shingles, option=option)
  if print_matrix:
    print_pairwise_matrix(matrix)
  
  mask = np.where(matrix <= threshold)

  PAIRS = []
  detected_suspect_shingles_to_exclude_semantics = set()
  for x, y in zip(mask[0], mask[1]):
    detected_suspect_shingles_to_exclude_semantics.add(x)
    pair = {
        'suspect_pos': x,
        'suspect_text': suspect_shingles[x],
        'candidate_pos': y,
        'candidate_text': candidate_shingles[y],
        'jaccard_distance': matrix[x][y]
    }
    PAIRS.append(pair)
  #print(detected_suspect_shingles_to_exclude_semantics)
  #Get list of shingle pairs to be submitted for semantic comparison
  mask_semant = np.where((threshold < matrix) & (matrix <= threshold_semant))
  ic("GIVE UNIQUE SUS POSITIONS THAT ARE LEFT INSTEAD OF PAIRS")
  PAIRS_SEMANTIC = []
  for x, y in zip(mask_semant[0], mask_semant[1]):
    if x not in detected_suspect_shingles_to_exclude_semantics:
      pair = {
          'suspect_pos': x,
          'candidate_pos': y,
          'jaccard_distance': matrix[x][y]
      }
      PAIRS_SEMANTIC.append(pair)
  #clean pairs for merging overlap
  #unique_suspect_positions = [pair['suspect_pos'] for pair in PAIRS]

  #percentage, detected_passages, number_passages = aggregate_report(unique_suspect_positions, suspect_shingles, suspect_text_length)
  _pairs = [(pair['suspect_pos'], pair['candidate_pos']) for pair in PAIRS]
  INSTANCES, percentage, granularity = aggregate_typical_subreport(_pairs, suspect_shingles, candidate_shingles, suspect_text_length, plag_type = PlagType.SYNTACT, skip_stride=True)

  PARAMETERS = {'n-gram_size': ngram_size, 'n-gram_stride': ngram_stride, 'threshold': threshold, 'percentage': percentage, 'granularity': granularity}

  #return PAIRS, PARAMETERS, PASSAGE_PAIRS, PAIRS_SEMANTIC
  return PARAMETERS, INSTANCES, PAIRS_SEMANTIC

In [ ]:
  ranges = [(x.posrange_suspect, x.posrange_candidate) for x in instances]
  ranges = merge_ranges(ranges)
  ranges

Calling

In [44]:
#pre_processing
suspect_text = samp_suspect[0]
candidate_text = samp_suspect[0]

suspect_shingles = shingles(suspect_text, ngram_size=15, stride=7)
candidate_shingles = shingles(candidate_text, ngram_size=15, stride=7)

suspect_text_length = len(re.findall(r'\w+', suspect_text))

#detect language

#jaccard_report
params, instances, pairs_semantic = jaccard_subreport(suspect_shingles, candidate_shingles, suspect_text_length, threshold = 0.65, threshold_semant=SEMANTIC_THRESHOLD)
ic("=========================== Jaccard")
ic(len(instances))
ic(len(pairs_semantic))
#fasttext_report = 
#def combined_subreport(suspect_file, candidate_file, params)

ic| 'GIVE UNIQUE SUS POSITIONS THAT ARE LEFT INSTEAD OF PAIRS'
ic| 'EXCLUDE SINGLE SUSPECT SHINGLES THAT APPEAR IN ANOTHER RANGE'
ic| 'found overlap'
ic| instances_length: 1093
ic| '=========================== Jaccard'
ic| len(instances): 1
ic| len(pairs_semantic): 0


[((0, 154), (0, 154)), ((0, 153), (1, 154)), ((1, 154), (0, 153)), ((43, 43), (45, 45)), ((45, 45), (43, 43)), ((69, 69), (78, 78)), ((78, 78), (69, 69)), ((133, 133), (135, 135)), ((135, 135), (133, 133))]
((0, 154), (0, 154))
Detected percentage = 99.45% (text = 1099 & detected = 1093)


0

In [46]:
len(instances)

1

In [47]:
params

{'granularity': 1,
 'n-gram_size': 15,
 'n-gram_stride': 7,
 'percentage': 99.45,
 'threshold': 0.65}

####Benchmarking Threshold

In [ ]:
thresholds = np.arange(.02, .5, 0.02)
PERCENTAGES = []
for threshold in thresholds:
  thresh = round(threshold, 2)
  pairs, params, portions = jaccard_global_report(samp_suspect[0], samp_candidate[0], threshold=thresh)
  item = {
      'threshold': thresh,
      'percentage': params['percentage'],
      'shingles': params['number_of_shingles']
  }
  PERCENTAGES.append(item)

PERCENTAGES

In [ ]:
%%time
%memit pairs, params, portions, pairs_semantic = jaccard_subreport(suspect_shingles, candidate_shingles, 1011, threshold = 0.65, threshold_semant=0.75)
#%memit pairs, params, portions = jaccard_subreport(suspect_shingles, candidate_shingles, 1011)

##Fasttext

In [51]:
def extract_fasttext_features(shingles: List[str], ft):
  '''
  Vectorize input strings into Fasttext features using a pre-trained & pre-loaded model {ft}
  {shingles} array of strings to vectorize
  {ft} pre-loaded Fasttext model
  -----
  %output: list of features, vectors of size XXXXXX
  '''
  features = []
  for i in shingles.keys():
    features.append(ft.get_sentence_vector(shingles[i].replace('\n', ' ')))
  return features


def fasttext_pairwise_cosine_distance(suspect_features, candidate_features):
  '''
  Generate pairwise cosine distance matrix (not similarity) of size len(suspects) x len(candidates)
  {suspect_features, candidate_features} arrays of Fasttext features vectors
  -----
  %output: array of arrays, pairwise distance matrix
  '''
  matrix = 1 - cosine_similarity(suspect_features, candidate_features)
  return matrix



In [57]:
def fasttext_subreport(suspect_shingles, candidate_shingles, suspect_pos_semantic, suspect_text_length, ft_model, threshold:float=0.1, ngram_size:int=15, ngram_stride:int=7, print_matrix=False):
  '''
  Generate global similarity detection report with Fasttext similarity 
  {suspect_text, candidate_text} standard string of text (not preprocessed)
  {ft_model} pre-loaded Fasttext model
  {threshold} value under which the cosine distance indicates a pair of strings to be similar. Smaller is more similar
  {ngram_size} as defined for [shingles] function
  {ngram_stride} as defined for [shingles] function
  {print_matrix} to plot the cosine pairwise distance matrix
  -----
  %output: 
  1- array of {
      suspect_pos:int, suspect_text:str, 
      candidate_pos:int, candidate_text:str
      cosine_distance:float
      }
  2- dict of input hyper-parameters + global detected plagiarism percentage & number of unique shingles that were highlighted (for granularity)
  '''
  ic("SUBSTITUTE COMPREHENSION WITH GENERATORS")

  ic("PATCH ME, I AM NOT OPTIMIZED, I EXCLUDE ALREADY DETECTED SHINGLES AFTER MATRIX CALCULATION")
  if (not isinstance(ngram_size, int)) or (not isinstance(ngram_stride, int)) or (threshold >= 1):
    raise Exception("Error, incorrect parameters for fasttext_subreport:\n- N-Gram size: {} (should be int [1;~])\n- Stride: {} (should be int [1;~])\n- Threshold: {} (should be float [.0;1.0[)".format(ngram_size, ngram_stride, threshold))
  if (len(max(suspect_shingles.values(), key=len)) > 400) or (len(max(candidate_shingles.values(), key=len)) > 400):
    raise Exception("Error, one or more input shingles is too large for Fasttext to parse")

  try: #To check for Fasttext loading
    ft_model.get_sentence_vector("Loading Fasttext")
  except:
    raise Exception("Fasttext model is not loaded")

  suspect_features = extract_fasttext_features(suspect_shingles, ft)
  candidate_features = extract_fasttext_features(candidate_shingles, ft)

  matrix = fasttext_pairwise_cosine_distance(suspect_features, candidate_features)
  mask = np.where(matrix < threshold)

  if print_matrix:
    print_pairwise_matrix(matrix)
  PAIRS = []
  for x, y in zip(mask[0], mask[1]):
    if x in suspect_pos_semantic:
      pair = {
          'suspect_pos': x,
          'suspect_text': suspect_shingles[x],
          'candidate_pos': y,
          'candidate_text': candidate_shingles[y],
          'cosine_distance': matrix[x][y]
      }
      PAIRS.append(pair)

  _pairs = [(pair['suspect_pos'], pair['candidate_pos']) for pair in PAIRS]
  INSTANCES, percentage, granularity = aggregate_typical_subreport(_pairs, suspect_shingles, candidate_shingles, suspect_text_length,  plag_type = PlagType.SEMANT, skip_stride=True)
  PARAMETERS = {'n-gram_size': ngram_size, 'n-gram_stride': ngram_stride, 'threshold': threshold, 'percentage': percentage, 'granularity': granularity}

  return PARAMETERS, INSTANCES

Loading a Fasttext model

In [ ]:
path = '/content/cc.en.300.bin'
try: 
  ft.get_sentence_vector("Loading Fasttext")
except:
  ic("Loading Fasttext")
  ft = fasttext.load_model(path)

Calling

In [ ]:
suspect_pos_semantic = [x['suspect_pos'] for x in pairs_semantic]
suspect_shingles_semantic = {x: suspect_shingles[x] for x in suspect_pos_semantic}
FASTTEXT_THRESHOLD

In [ ]:
%%time
%memit params, instances = fasttext_subreport(suspect_shingles, candidate_shingles, suspect_pos_semantic, suspect_text_length, ft_model=ft, threshold = FASTTEXT_THRESHOLD, print_matrix=False)

ic| 'SUBSTITUTE COMPREHENSION WITH GENERATORS'
ic| "PATCH ME, I AM NOT OPTIMIZED, I EXCLUDE ALREADY DETECTED SHINGLES AFTER MATRIX CALCULATION": ('PATCH ME, I AM NOT OPTIMIZED, I EXCLUDE ALREADY DETECTED SHINGLES AFTER '
                                                                                                   'MATRIX CALCULATION')
ic| 'got suspect features'
ic| 'got candidate features'
ic| 'got matrix'
ic| 'got pairs'
ic| 'found overlap'
ic| instances_length: 10487
ic| 'aggregated report'


Detected percentage = 99.97% (text = 10490 & detected = 10487)
peak memory: 8323.24 MiB, increment: 0.77 MiB
CPU times: user 23.2 s, sys: 670 ms, total: 23.9 s
Wall time: 23.6 s


In [ ]:
ic(pairs_semantic[:2])
ic(len(suspect_shingles))
ic(len(instances))

ic| pairs_semantic[:2]: [{'candidate_pos': 1, 'jaccard_distance': 0.75, 'suspect_pos': 0},
                         {'candidate_pos': 8, 'jaccard_distance': 0.6956521739130435, 'suspect_pos': 0}]
ic| len(suspect_shingles): 155
ic| len(instances): 13


13

In [ ]:
portions

Benchmarking Threshold

In [ ]:
thresholds = np.arange(.02, .16, 0.01)
PERCENTAGES = []
for threshold in thresholds:
  thresh = round(threshold, 2)
  pairs, params, portions = fasttext_global_report(samp_suspect[1], samp_candidate[0], ft_model=ft, threshold=thresh)
  item = {
      'threshold': thresh,
      'percentage': params['percentage'],
      'shingles': params['number_of_shingles']
  }
  PERCENTAGES.append(item)

##GridSearch for Hyperparameters selection

#Union Hangar

In [35]:
NGRAM_SIZE_SHINGLES = 15
STRIDE_SHINGLES = 7
SKIP_TOLERANCE = int(NGRAM_SIZE_SHINGLES/STRIDE_SHINGLES)

JACCARD_THRESHOLD = 0.25
SEMANTIC_THRESHOLD = 0.99
FASTTEXT_THRESHOLD = 0.11

In [144]:
def calculate_suspect_length_from_ranges(ranges: List[Tuple[int, int]], ngram_size: int=NGRAM_SIZE_SHINGLES, ngram_stride: int=STRIDE_SHINGLES):
  #exclude all overlapping & duplicate ranges
  suspect_ranges = list(dict.fromkeys(ranges)) #eliminating duplicates

  suspect_ranges = merge_score_ranges(suspect_ranges)
  total_length = 0
  for range in suspect_ranges:
    number_strides = range[1] - range[0]
    length = ngram_size + ngram_stride*number_strides
    total_length += length
  return total_length

In [133]:
def aggregate_typical_subreport(suspect_candidate_pairs_positions, suspect_shingles, candidate_shingles, suspect_text_length:int, plag_type:PlagType, skip_stride=True, ngram_stride=7, ngram_size=15):
    '''
    Calculate total detected percentage after reducing granularity
      {suspect_candidate_pairs_positions} list of pair positions (sus, can) of the highlighted shingles from suspect and candidate
      {suspect_shingles, candidate_shingles} complete list of shingles of the source texts (not just the highlighted ones)
      {skip_stride} since shingles overlap, this options will force to combine non consecutive shingles that are close enough, eith distance <= ngram_size/ngram_stride
                If false, only true consecutive shingles will be combined (may increase granularity)
      {ngram_size} as defined for [shingles] function
      {ngram_stride} as defined for [shingles] function
      {exclude_single_ngrams} to exclude single orphans if ngrams size is small enough (SMALL_NGRAM_THRESHOLD_COMMON_SUBSTRING)
      {source_text_length} length of the original text (not shingled)

    -----
    %output: 
      1- array of {position_range:(start, end), passage:str}
      2- total size of the passages (for percentage calculation, will be removed)
    '''
    #unique_positions = list(dict.fromkeys(highlighted_shingles_positions))

    instances, granularity = merge_shingles(suspect_candidate_pairs_positions, suspect_shingles, candidate_shingles, plag_type = plag_type, skip_stride=skip_stride, ngram_stride=ngram_stride, ngram_size=ngram_size)
    
    highlighted_suspect_ranges = [instance.posrange_suspect for instance in instances] #extracting suspect ranges only 
    highlighted_suspect_length = calculate_suspect_length_from_ranges(highlighted_suspect_ranges)
    
    score = round(100*highlighted_suspect_length/suspect_text_length, 2)

    print("Detected percentage = {}% (text = {} & detected = {})".format(score, suspect_text_length, highlighted_suspect_length))

    return instances, score, granularity

#-------------------------

In [ ]:
#Working on (this is the code down there)
def detect_plag_subreport(suspect_text, candidate_text, ft):
  '''Super Caller function for fastText & Jaccard subreporting for a single candidate/suspect pair.'''
  "Code starter in used to call Jaccard"
  ic("use suspect and candidate file structures instead of fulltext (suspect also contains language, which removes the need to test it again each time)")
  ic("FEED ME PRELOADED MODEL")
  #pre_processing
  suspect_shingles = shingles(suspect_text, ngram_size=15, stride=7)
  candidate_shingles = shingles(candidate_text, ngram_size=15, stride=7)

  #detect language
  suspect_text_length = len(re.findall(r'\w+', suspect_text))
  suspect_language = detect_language(suspect_text)

  #jaccard_report
  ic("=========================== Jaccard")
  params_jaccard, instances_jaccard, pairs_semantic = jaccard_subreport(suspect_shingles, candidate_shingles, suspect_text_length, threshold = JACCARD_THRESHOLD, threshold_semant=SEMANTIC_THRESHOLD)

  ic("=========================== fastText")
  suspect_pos_semantic = [x['suspect_pos'] for x in pairs_semantic]
  #suspect_shingles_semantic = {x: suspect_shingles[x] for x in suspect_pos_semantic}
  params_fasttext, instances_fasttext = fasttext_subreport(suspect_shingles, candidate_shingles, suspect_pos_semantic, suspect_text_length, ft_model=ft, threshold = FASTTEXT_THRESHOLD, print_matrix=False)

  #fasttext_report = 
  #def combined_subreport(suspect_file, candidate_file, params)
  

In [152]:
  suspect_text = samp_suspect_long[0]
  candidate_text = samp_candidate_long[0]
  

  path = '/content/cc.en.300.bin'
  try: 
    ft.get_sentence_vector("Loading Fasttext")
  except:
    ic("Loading Fasttext")
    try:
      ft = fasttext.load_model(path)
    except:
      raise Exception(f"Error loading fastText model for language {'fr'}")

  '''Super Caller function for fastText & Jaccard subreporting for a single candidate/suspect pair.'''
  "Code starter in used to call Jaccard"
  ic("use suspect and candidate file structures instead of fulltext (suspect also contains language, which removes the need to test it again each time)")
  ic("FEED ME PRELOADED MODEL")
  #pre_processing
  suspect_shingles = shingles(suspect_text, ngram_size=15, stride=7)
  candidate_shingles = shingles(candidate_text, ngram_size=15, stride=7)

  #detect language
  suspect_text_length = len(re.findall(r'\w+', suspect_text))
  suspect_language = detect_language(suspect_text)
  ic(suspect_language)

  #jaccard_subreport
  ic("=========================== Jaccard")
  params_syntact, instances_syntact, pairs_semantic = jaccard_subreport(suspect_shingles, candidate_shingles, suspect_text_length, threshold = JACCARD_THRESHOLD, threshold_semant=SEMANTIC_THRESHOLD)

  #fasttext_subreport
  ic("=========================== fastText")
  suspect_pos_semantic = [x['suspect_pos'] for x in pairs_semantic]
  suspect_pos_semantic = list(dict.fromkeys(suspect_pos_semantic)) #to remove duplicates
  #suspect_pos_semantic = [x for x in range(2500)]
  #suspect_shingles_semantic = {x: suspect_shingles[x] for x in suspect_pos_semantic}
  params_semant, instances_semant = fasttext_subreport(suspect_shingles, candidate_shingles, suspect_pos_semantic, suspect_text_length, ft_model=ft, threshold = FASTTEXT_THRESHOLD, print_matrix=False)


  #merge the two instances to have a single global score
  instances = instances_semant + instances_syntact
  instances = sorted(instances, key = lambda i: i.posrange_suspect[0])

  ranges = [x.posrange_suspect for x in instances]

  total_suspect_highlighted_length = calculate_suspect_length_from_ranges(ranges)
  global_score = round(100*total_suspect_highlighted_length/suspect_text_length, 2)
  semant_score = params_semant['percentage']
  syntact_score = params_syntact['percentage']

  detection_scores = DetectionScores(syntact_score, semant_score, global_score)
  #merge the sub reports into a single subreport 
  ic(detection_scores)

ic| "use suspect and candidate file structures instead of fulltext (suspect also contains language, which removes the need to test it again each time)": ('use suspect and candidate file structures instead of fulltext (suspect also '
                                                                                                                                                          'contains language, which removes the need to test it again each time)')
ic| 'FEED ME PRELOADED MODEL'
ic| suspect_language: <Language.ENGLISH: 'en'>
ic| '=========================== Jaccard'
ic| 'GIVE UNIQUE SUS POSITIONS THAT ARE LEFT INSTEAD OF PAIRS'
ic| 'EXCLUDE SINGLE SUSPECT SHINGLES THAT APPEAR IN ANOTHER RANGE'
ic| '=========================== fastText'


Detected percentage = 7.7% (text = 10490 & detected = 808)


ic| 'SUBSTITUTE COMPREHENSION WITH GENERATORS'
ic| "PATCH ME, I AM NOT OPTIMIZED, I EXCLUDE ALREADY DETECTED SHINGLES AFTER MATRIX CALCULATION": ('PATCH ME, I AM NOT OPTIMIZED, I EXCLUDE ALREADY DETECTED SHINGLES AFTER '
                                                                                                   'MATRIX CALCULATION')
ic| 'EXCLUDE SINGLE SUSPECT SHINGLES THAT APPEAR IN ANOTHER RANGE'
ic| 'found overlap'
ic| detection_scores: DetectionScores(syntac_score=7.7, semant_score=93.63, global_score=94.47)


Detected percentage = 93.63% (text = 10490 & detected = 9822)


DetectionScores(syntac_score=7.7, semant_score=93.63, global_score=94.47)

In [130]:
def checkstate_merge_score_ranges(ranges: List[Tuple[int, int]])-> bool:
  for i in range(len(ranges) - 1):
    range_sus = ranges[i]
    range_sus_next = ranges[i+1]
    if (range_sus[1] >= range_sus_next[0]):
      return True
  return False

#merge_ranges_overlap
def merge_score_ranges(ranges: List[Tuple[int, int]])-> List[Tuple[int, int]]:
  if len(ranges) > 1:
    
    non_overlaping_ranges = []
    i = 0
    while i in range(len(ranges) - 1):
      range_sus = ranges[i]
      range_sus_next = ranges[i+1]
      if (range_sus[1] >= range_sus_next[0]):
        new_range_sus = (min(range_sus[0], range_sus_next[0]), max(range_sus[1], range_sus_next[1]))
        non_overlaping_ranges.append(new_range_sus)
        i += 1 #skip next element because it was merged
      else:
        non_overlaping_ranges.append(range_sus)
        if i == len(ranges) - 2: #to deal with last element
          non_overlaping_ranges.append(range_sus_next)
      i += 1
    if checkstate_merge_score_ranges(non_overlaping_ranges):
      non_overlaping_ranges = merge_score_ranges(non_overlaping_ranges)
    return non_overlaping_ranges
  else:
    return ranges

In [ ]:
def merge_syntac_semant_instances(syntac_instances:List[PlagInstance], semant_instances:List[PlagInstance]) -> List[PlagInstance]:
  '''Aggregate instances from the typical subreports
  Merge everything
  May call the aggregate_typical_scores func or do its job in here'''

  #start with semantic instances, fix them as the baseline and consider that everything that is syntactic, is also sematically detected
  #break the semantic instances to leave only ranges that do not overlap with syntactical ones

  
  ...

In [ ]:
@dataclass(frozen=True)
class PlagInstance:
  posrange_suspect: (int, int)
  posrange_candidate: (int, int)
  passsage_suspect: str
  passsage_candidate: str
  plag_type: PlagType

@dataclass(frozen=True)
class CanSource:
  can_name: str
  can_url: str
  can_id: str
  can_type: CanType 
  can_fulltext: str

@dataclass(frozen=True)
class Suspect:
  sus_name: str
  sus_id: str
  sus_lang: Language
  sus_fulltext: str

In [ ]:
#testing ranges merging
semantics = [PlagInstance(posrange_suspect=(0, 18), posrange_candidate=(0, 17), passsage_suspect='the significance of export platform fdi is reported in a think about by hanson', passsage_candidate='the significance of export platform fdi is reported in a think about by hanson', plag_type=PlagType.SEMANT),
PlagInstance(posrange_suspect=(24, 74), posrange_candidate=(23, 71), passsage_suspect='time required to reach the center lesson the promoting of extravagance has contributed altogether', passsage_candidate='the significance of export platform fdi is reported in a think about by hanson', plag_type=PlagType.SEMANT)]

syntactics = [PlagInstance(posrange_suspect=(3, 9), posrange_candidate=(0, 17), passsage_suspect='the significance of export platform fdi is reported in a think about by hanson', passsage_candidate='the significance of export platform fdi is reported in a think about by hanson', plag_type=PlagType.SYNTACT),
PlagInstance(posrange_suspect=(27, 76), posrange_candidate=(23, 71), passsage_suspect='time required to reach the center lesson the promoting of extravagance has contributed altogether', passsage_candidate='the significance of export platform fdi is reported in a think about by hanson', plag_type=PlagType.SYNTACT)]

instances = semantics + syntactics
ic(instances[3].posrange_suspect[0])

instances = ic(sorted(instances, key = lambda i: i.posrange_suspect[0]))
ranges = [(x.posrange_suspect, x.posrange_candidate) for x in instances]
ranges


ranges = merge_ranges(ranges)
ranges


ic| instances[3].posrange_suspect[0]: 27
ic| sorted(instances, key = lambda i: i.posrange_suspect[0]): [PlagInstance(posrange_suspect=(0, 18), posrange_candidate=(0, 17), passsage_suspect='the significance of export platform fdi is reported in a think about by hanson', passsage_candidate='the significance of export platform fdi is reported in a think about by hanson', plag_type=<PlagType.SEMANT: 2>),
                                                               PlagInstance(posrange_suspect=(3, 9), posrange_candidate=(0, 17), passsage_suspect='the significance of export platform fdi is reported in a think about by hanson', passsage_candidate='the significance of export platform fdi is reported in a think about by hanson', plag_type=<PlagType.SYNTACT: 0>),
                                                               PlagInstance(posrange_suspect=(24, 74), posrange_candidate=(23, 71), passsage_suspect='time required to reach the center lesson the promoting of extravagance has contribu

[((0, 18), (0, 17)), ((24, 76), (23, 71))]

## Report outputing & % counting

#Streamlit Serving

In [ ]:
#GridSearch benchmarking
from itertools import product
a = ['a', 'b', 'c']
one = [1, 2, 3]
d = ['d', 'e', 'f']

for x, y, z in product(a, one, d):
  print(x, y, z)


a 1 d
a 1 e
a 1 f
a 2 d
a 2 e
a 2 f
a 3 d
a 3 e
a 3 f
b 1 d
b 1 e
b 1 f
b 2 d
b 2 e
b 2 f
b 3 d
b 3 e
b 3 f
c 1 d
c 1 e
c 1 f
c 2 d
c 2 e
c 2 f
c 3 d
c 3 e
c 3 f


#Legacy code

In [ ]:
@dataclass(frozen=True)
class DetectionScores:
  """
  A class to hold detection scores registered after a comparison operation with a single candidate, or the aggregation of multiple reports.
  
  Attributes:
      syntac_score:
          Global syntactic similarity between suspect file and candidates (one if SubReport, multiple if PlagReport)
          syntac_score = size(suspect text) / sum(size (suspect segments that appear as a member in pairs with Jaccard similarity < threshold))
      semant_score:
          Global semantic similarity between suspect file and candidates (one if SubReport, multiple if PlagReport)
          semant_score= size(suspect text) / sum(size (suspect segments that appear as a member in pairs with FastText cosine similarity < threshold))
      global_score:
          Global similarity, calculated as sum of syntac_score + semant_score
  """
  syntac_score: float
  semant_score:float
  global_score: float = field(init=False)
  def __post_init__(self):
    object.__setattr__(self, 'global_score', (self.syntac_score + self.semant_score))

In [ ]:
def fuse_consecutive_shingles(positions:List[int], text_shingles, skip_stride=True, ngram_stride:int=7, ngram_size:int=15):
  #TRANSFORM ME FOR BOTH SUSPECT AND CANDIDATE OR GENERALIZE ME AND USE ME TWICE
  '''
  Parse and combine consecutive highlighted shingles based on their position in the source text
  {positions} list of positions of the highlighted shingles
  {text_shingles} complete list of shingles of the source text (not just the highlighted ones)
  {skip_stride} since shingles overlap, this options will force to combine non consecutive shingles that are close enough, eith distance <= ngram_size/ngram_stride
                If false, only true consecutive shingles will be combined (may increase granularity)
  {ngram_size} as defined for [shingles] function
  {ngram_stride} as defined for [shingles] function
  -----
  %output: 
  1- array of {position_range:(start, end), passage:str}
  2- total size of the passages (for percentage calculation, will be removed)
  '''
  #build chains of consecutive, or forgivingly consecutive shingles
  cpt = 0
  suites = defaultdict(list)
  cache = dict()
  for i in positions:
    if (i-1) in positions:
      #print("Flush")
      index = cache[i-1]
      suites[index].append(i)
      cache[i] = index
    elif skip_stride and any(x in positions for x in range(i-int(ngram_size/ngram_stride), i-1)): #if word in between was modified, combine suites if single word is missing (i.e if distance == STRIDE)
      #print("Close Flush")
      for x in range(i-ngram_stride, i-1): #to find which one satisfied the condition first (smaller)
        if x in positions:
          index = cache[x]
          suites[index].append(i)
          cache[i] = index
          break
    else:
      suites[cpt].append(i)
      cache[i] = cpt
      cpt += 1
    #transform back into text
  result = []
  total_size = 0
  for suite in suites.values():
    pos = (suite[0], suite[-1])
    pos_range = [y for y in range(pos[0]+1,pos[1]+1)]
    passage = text_shingles[suite[0]]
    
    #adding only the last word of the shingle, regardless of stride size
    passage += " " + " ".join(" ".join(text_shingles[x].split()[-ngram_stride:]) for x in pos_range)

    item = {
        'passage_suspect': passage,
        'pos_suspect': (suite[0], suite[-1]),
    }
    result.append(item)
    total_size += len(re.findall(r'\w+', passage))
    #ic(suite, len(passage), total_size, passage)
  return result, total_size

In [ ]:
  def aggregate_report(highlighted_shingles_positions, input_shingles, source_text_length:int, skip_stride=True, ngram_stride=7, ngram_size=15):
    '''
    Calculate total detected percentage after reducing granularity
    {highlighted_shingles_positions} array of int, list of postions
    {input_shingles} as defined for [shingles] function
    {source_text_length} length of the original text (not shingled)
    {skip_stride} as defined for [fuse_consecutive_shingles] function
    {ngram_size} as defined for [shingles] function
    {ngram_stride} as defined for [shingles] function
    -----
    %output: 
    1- array of {position_range:(start, end), passage:str}
    2- total size of the passages (for percentage calculation, will be removed)
    '''
    unique_positions = list(dict.fromkeys(highlighted_shingles_positions))
    
    passages, passages_length = fuse_consecutive_shingles(unique_positions, input_shingles, skip_stride=skip_stride, ngram_stride=ngram_stride, ngram_size=ngram_size)
    
    percentage = (float(passages_length)/source_text_length) * 100
    percentage = round(percentage, 2)
    
    print("Detected percentage = {}% (text = {} & detected = {})".format(percentage, source_text_length, passages_length))

    return percentage, passages, len(unique_positions)

#To-Do

In [15]:
def aggregate_scores(self, subreports: List[SubReport]) -> DetectionScores:
  '''
  Calculate global similarity scores based on sub-reports scores.

  If a passage appears in both a semantic and syntactic comparison, priority is for syntactic (until user excludes a source).

  :param subreports: list of all subreports that were generated by comparing suspect file to all candidate files.
  :returns: a DetectionScores unit similar to those in subreports, but is an aggregation of all of them.
  :raises keyError: raises an exception
  '''
  ic("FIX ME I'M UNCOMPLETE AND SHOULD AGGREGATE BASED ON OVERLAP BETWEEN CANDIDATES TOO")
  #%%%%%%%%%%%%%%%&&&&&&&&&&&&&&&&%%%%%%%%%%%%%%%&&&&&&&&&&&&&&&&%%%%%%%%%%%%%%%&&&&&&&&&&&&&&&&
  syntac_scores = [x.scores.syntac_score for x in subreports]
  global_syntac_score = sum(syntac_scores)

  semant_scores = [x.scores.semant_score for x in subreports]
  global_semant_score = sum(semant_scores)

  scores = DetectionScores(global_syntac_score, global_semant_score)
  return scores

In [ ]:



def calculate_subreport_score():
  ...
def from_shingle_ranges_to_text_offsets():
  ...
def aggregate_typical_detection_scores(syntac_instances: List[PlagInstance], semant_instances: List[PlagInstance]):
  '''Logically aggegate detection scores from typical subreports. Give priority to syntactic similarity.'''
  ...
def aggregate_detection_scores(scores: List[DetectionScores]):
  '''Logically aggegate detection scores across all subreports. should be soft and not delete any instances or subreports score as the need to recalculate scores with a different set of scores may arise'''
  ...

def aggregate_typical_report():
  '''Combine Jaccard + fastText reports'''
  ...


def detect_plag_report():
  '''Aggregate detect_plag_subreports into a single report'''
def preprocess_stopwords(shingle: str):
  '''Removes stopwords from a single string'''
  ...

def load_ft_model(language: Language):
  ...

def jsonify_report():
  ...

def scrape_webpage(url: str) -> str:
  ...
